# TrenchRipper Master Notebook

## Introduction

This notebook contains the entire `TrenchRipper` pipline, divided into simple steps. This pipline is ideal for Mother <br>Machine image data where cells possess fluorescent segmentation markers. Segmentation on phase or brightfield data <br>is being developed, but is still an experimental feature.

The steps in this pipeline are as follows:
1. Extracting your Mother Machine data (.nd2) into hdf5 format
2. Identifying and cropping individual trenches into kymographs
3. Segmenting cells with a fluorescent marker
4. Determining lineages and object properties

In each step, the user will dynamically specify parameters using a series of interactive diagnostics on their dataset. <br>Following this, a parameter file will be written to disk and then used to deploy a parallel computation on the <br>dataset, either locally or on a SLURM cluster.


This is intended as an end-to-end solution to analyzing Mother Machine data. As such, **it is not trivial to plug data <br>directly into intermediate steps**, as it will lack the correct formatting and associated metadata. A notable <br>exception to this is using another program to segment data. The library references binary segmentation masks using <br>only metadata derived from their associated kymographs. As such, it is possible to generate segmentations on these <br>kymographs elsewhere and place them into the segmentation data path to have `TrenchRipper` act on those <br>segmentations instead. More on this in the segmentation section...

#### Imports

Run this section to import all relavent packages and libraries used in this notebook. You must run this everytime you open a new python kernel.

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

#### Specify Paths

Begin by defining the directory in which all processing will be done, as well as the initial nd2 file we will be <br>processing. This line should be run everytime you open a new python kernel.

The format should be: `headpath = "/path/to/folder"` and `nd2file = "/path/to/file.nd2"`

For example:
```
headpath = "/n/scratch2/de64/2019-05-31_validation_data"
nd2file = "/n/scratch2/de64/2019-05-31_validation_data/Main_Experiment.nd2"
```

Ideally, these files should be placed in a storage location with relatively fast I/O

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/gfp/"
# hdf5inputpath = "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/run/"
nd2file = "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/GFP001.nd2"

In [ ]:
import h5py
import os
import shutil
import copy
import h5py_cache
import tifffile
import pickle as pkl
import numpy as np
import pandas as pd
import ipywidgets as ipyw

from nd2reader import ND2Reader
from tifffile import imsave, imread
from paulssonlab.deaton.trenchripper.trenchripper.utils import (
    pandas_hdf5_handler,
    writedir,
)
from parse import compile


class marlin_extractor:
    def __init__(
        self,
        hdf5inputpath,
        headpath,
        tpts_per_file=100,
        parsestr="fov={fov:d}_config={channel}_t={timepoints:d}.hdf5",
        metaparsestr="metadata_t={timepoint:d}.hdf5",
        zero_base_keys=["timepoints"],
    ):  # note this chunk size has a large role in downstream steps...make sure is less than 1 MB
        """Utility to import hdf5 format files from MARLIN Runs.

        Attributes:
            headpath (str): base directory for data analysis
            tiffpath (str): directory where tiff files are located
            metapath (str): metadata path
            hdf5path (str): where to store hdf5 data
            tpts_per_file (int): number of timepoints to put in each hdf5 file
            parsestr (str): format of filenames from which to extract metadata (using parse library)
        """
        self.hdf5inputpath = hdf5inputpath
        self.headpath = headpath
        self.metapath = self.headpath + "/metadata.hdf5"
        self.hdf5path = self.headpath + "/hdf5"
        self.tpts_per_file = tpts_per_file
        self.parsestr = parsestr
        self.metaparsestr = metaparsestr
        self.zero_base_keys = zero_base_keys

        self.organism = ""
        self.microscope = ""
        self.notes = ""

    def get_metadata(
        self,
        hdf5inputpath,
        parsestr="fov={fov:d}_config={channel}_t={timepoints:d}.hdf5",
        metaparsestr="metadata_t={timepoint:d}.hdf5",
        zero_base_keys=["timepoints"],
    ):
        parser = compile(parsestr)
        parse_keys = [
            item.split("}")[0].split(":")[0] for item in parsestr.split("{")[1:]
        ] + ["image_paths"]

        exp_metadata = {}
        fov_metadata = {key: [] for key in parse_keys}

        hdf5_files = []
        metadata_files = []
        for root, _, files in os.walk(hdf5inputpath):
            hdf5_files.extend(
                [
                    os.path.join(root, f)
                    for f in files
                    if "config" in os.path.splitext(f)[0]
                ]
            )
            metadata_files.extend(
                [
                    os.path.join(root, f)
                    for f in files
                    if "metadata" in os.path.splitext(f)[0]
                ]
            )

        with h5py.File(hdf5_files[0], "r") as infile:
            hdf5_shape = infile["data"].shape
        exp_metadata["height"] = hdf5_shape[0]
        exp_metadata["width"] = hdf5_shape[1]
        #     exp_metadata['pixel_microns'] = tags['65326']

        for f in hdf5_files:
            match = parser.search(f)
            # ignore any files that don't match the regex
            if match is not None:
                # Add to dictionary
                fov_frame_dict = match.named
                for key, value in fov_frame_dict.items():
                    fov_metadata[key].append(value)
                fov_metadata["image_paths"].append(f)

        for zero_base_key in zero_base_keys:
            if 0 not in fov_metadata[zero_base_key]:
                fov_metadata[zero_base_key] = [
                    item - 1 for item in fov_metadata[zero_base_key]
                ]

        channels = list(set(fov_metadata["channel"]))
        exp_metadata["channels"] = channels
        exp_metadata["num_fovs"] = len(set(fov_metadata["fov"]))
        exp_metadata["frames"] = list(set(fov_metadata["timepoints"]))
        exp_metadata["num_frames"] = len(exp_metadata["frames"])
        exp_metadata["pixel_microns"] = 0.16136255757596  ##hack assuming ti5 40x
        fov_metadata = pd.DataFrame(fov_metadata)
        fov_metadata = fov_metadata.set_index(["fov", "timepoints"]).sort_index()

        output_fov_metadata = []
        step = len(channels)
        for i in range(0, len(fov_metadata), step):
            rows = fov_metadata[i : i + step]
            channel_path_entry = dict(zip(rows["channel"], rows["image_paths"]))
            fov_entry = rows.index.get_level_values("fov").unique()[0]
            timepoint_entry = rows.index.get_level_values("timepoints").unique()[0]
            fov_metadata_entry = {
                "fov": fov_entry,
                "timepoints": timepoint_entry,
                "channel_paths": channel_path_entry,
            }
            output_fov_metadata.append(fov_metadata_entry)
        fov_metadata = pd.DataFrame(output_fov_metadata).set_index(
            ["fov", "timepoints"]
        )

        metaparser = compile(metaparsestr)
        meta_df_out = []
        for metadata_file in metadata_files:
            match = metaparser.search(metadata_file)
            if match is not None:
                timepoint = match.named["timepoint"]
                meta_df = pd.read_hdf(metadata_file)
                meta_df["timepoints"] = timepoint
                meta_df_out.append(meta_df)
        meta_df_out = pd.concat(meta_df_out)
        if 0 not in meta_df_out["timepoints"].unique().tolist():
            meta_df_out["timepoints"] = meta_df_out["timepoints"] - 1
        meta_df_out = meta_df_out.groupby(["fov", "timepoints"], as_index=False)
        meta_df_out = meta_df_out.apply(lambda x: x[0:1])
        meta_df_out = meta_df_out.set_index(["fov", "timepoints"], drop=True)
        fov_metadata = fov_metadata.join(meta_df_out)

        return exp_metadata, fov_metadata

    def assignidx(self, fov_metadata):
        numfovs = len(fov_metadata.index.get_level_values("fov").unique().tolist())
        timepoints_per_fov = len(
            fov_metadata.index.get_level_values("timepoints").unique().tolist()
        )

        files_per_fov = (timepoints_per_fov // self.tpts_per_file) + 1
        remainder = timepoints_per_fov % self.tpts_per_file
        ttlfiles = numfovs * files_per_fov
        fov_file_idx = np.repeat(list(range(files_per_fov)), self.tpts_per_file)[
            : -(self.tpts_per_file - remainder)
        ]
        file_idx = np.concatenate(
            [fov_file_idx + (fov_idx * files_per_fov) for fov_idx in range(numfovs)]
        )
        fov_img_idx = np.repeat(
            np.array(list(range(self.tpts_per_file)))[np.newaxis, :],
            files_per_fov,
            axis=0,
        )
        fov_img_idx = fov_img_idx.flatten()[: -(self.tpts_per_file - remainder)]
        img_idx = np.concatenate([fov_img_idx for fov_idx in range(numfovs)])

        fov_idx = np.repeat(list(range(numfovs)), timepoints_per_fov)
        timepoint_idx = np.repeat(
            np.array(list(range(timepoints_per_fov)))[np.newaxis, :], numfovs, axis=0
        ).flatten()

        outdf = copy.deepcopy(fov_metadata)
        outdf["File Index"] = file_idx
        outdf["Image Index"] = img_idx
        return outdf

    def writemetadata(self, t_range=None, fov_list=None):

        exp_metadata, fov_metadata = self.get_metadata(
            self.hdf5inputpath,
            parsestr=self.parsestr,
            zero_base_keys=self.zero_base_keys,
        )

        if t_range is not None:
            exp_metadata["frames"] = exp_metadata["frames"][t_range[0] : t_range[1] + 1]
            exp_metadata["num_frames"] = len(exp_metadata["frames"])
            fov_metadata = fov_metadata.loc[
                pd.IndexSlice[:, slice(t_range[0], t_range[1])], :
            ]  # 4 -> 70

        if fov_list is not None:
            fov_metadata = fov_metadata.loc[list(fov_list)]
            exp_metadata["fields_of_view"] = list(fov_list)

        self.chunk_shape = (1, exp_metadata["height"], exp_metadata["width"])
        chunk_bytes = 2 * np.multiply.accumulate(np.array(self.chunk_shape))[-1]
        self.chunk_cache_mem_size = 2 * chunk_bytes
        exp_metadata["chunk_shape"], exp_metadata["chunk_cache_mem_size"] = (
            self.chunk_shape,
            self.chunk_cache_mem_size,
        )
        exp_metadata["Organism"], exp_metadata["Microscope"], exp_metadata["Notes"] = (
            self.organism,
            self.microscope,
            self.notes,
        )
        self.meta_handle = pandas_hdf5_handler(self.metapath)

        assignment_metadata = self.assignidx(fov_metadata)
        assignment_metadata.astype({"File Index": int, "Image Index": int})

        self.meta_handle.write_df("global", assignment_metadata, metadata=exp_metadata)

    def read_metadata(self):
        writedir(self.hdf5path, overwrite=True)
        self.writemetadata()
        metadf = self.meta_handle.read_df("global", read_metadata=True)
        self.metadata = metadf.metadata
        metadf = metadf.reset_index(inplace=False)
        metadf = metadf.set_index(
            ["File Index", "Image Index"], drop=True, append=False, inplace=False
        )
        self.metadf = metadf.sort_index()

    def set_params(self, fov_list, t_range, organism, microscope, notes):
        self.fov_list = fov_list
        self.t_range = t_range
        self.organism = organism
        self.microscope = microscope
        self.notes = notes

    def inter_set_params(self):
        self.read_metadata()
        t0, tf = (self.metadata["frames"][0], self.metadata["frames"][-1])
        available_fov_list = self.metadf["fov"].unique().tolist()
        selection = ipyw.interactive(
            self.set_params,
            {"manual": True},
            fov_list=ipyw.SelectMultiple(options=available_fov_list),
            t_range=ipyw.IntRangeSlider(
                value=[t0, tf],
                min=t0,
                max=tf,
                step=1,
                description="Time Range:",
                disabled=False,
            ),
            organism=ipyw.Textarea(
                value="",
                placeholder="Organism imaged in this experiment.",
                description="Organism:",
                disabled=False,
            ),
            microscope=ipyw.Textarea(
                value="",
                placeholder="Microscope used in this experiment.",
                description="Microscope:",
                disabled=False,
            ),
            notes=ipyw.Textarea(
                value="",
                placeholder="General experiment notes.",
                description="Notes:",
                disabled=False,
            ),
        )
        display(selection)

    def extract(self, dask_controller, retries=1):
        dask_controller.futures = {}

        self.writemetadata(t_range=self.t_range, fov_list=self.fov_list)
        metadf = self.meta_handle.read_df("global", read_metadata=True)
        self.metadata = metadf.metadata
        metadf = metadf.reset_index(inplace=False)
        metadf = metadf.set_index(
            ["File Index", "Image Index"], drop=True, append=False, inplace=False
        )
        self.metadf = metadf.sort_index()

        def writehdf5(fovnum, num_entries, timepoint_list, file_idx):
            y_dim = self.metadata["height"]
            x_dim = self.metadata["width"]
            filedf = self.metadf.loc[file_idx].reset_index(inplace=False)
            filedf = filedf.set_index(
                ["timepoints"], drop=True, append=False, inplace=False
            )
            filedf = filedf.sort_index()

            with h5py_cache.File(
                self.hdf5path + "/hdf5_" + str(file_idx) + ".hdf5",
                "w",
                chunk_cache_mem_size=self.chunk_cache_mem_size,
            ) as h5pyfile:
                for i, channel in enumerate(self.metadata["channels"]):
                    hdf5_dataset = h5pyfile.create_dataset(
                        str(channel),
                        (num_entries, y_dim, x_dim),
                        chunks=self.chunk_shape,
                        dtype="uint16",
                    )
                    for j in range(len(timepoint_list)):
                        frame = timepoint_list[j]
                        entry = filedf.loc[frame]["channel_paths"]
                        file_path = entry[channel]
                        with h5py_cache.File(file_path, "r") as infile:
                            img = infile["data"][:]
                        hdf5_dataset[j, :, :] = img
            return "Done."

        file_list = self.metadf.index.get_level_values("File Index").unique().values
        num_jobs = len(file_list)
        random_priorities = np.random.uniform(size=(num_jobs,))

        for k, file_idx in enumerate(file_list):
            priority = random_priorities[k]
            filedf = self.metadf.loc[file_idx]

            fovnum = filedf[0:1]["fov"].values[0]
            num_entries = len(filedf.index.get_level_values("Image Index").values)
            timepoint_list = filedf["timepoints"].tolist()

            future = dask_controller.daskclient.submit(
                writehdf5,
                fovnum,
                num_entries,
                timepoint_list,
                file_idx,
                retries=retries,
                priority=priority,
            )
            dask_controller.futures["extract file: " + str(file_idx)] = future

        extracted_futures = [
            dask_controller.futures["extract file: " + str(file_idx)]
            for file_idx in file_list
        ]
        pause_for_extract = dask_controller.daskclient.gather(
            extracted_futures, errors="skip"
        )

        futures_name_list = ["extract file: " + str(file_idx) for file_idx in file_list]
        failed_files = [
            futures_name_list[k]
            for k, item in enumerate(extracted_futures)
            if item.status != "finished"
        ]
        failed_file_idx = [int(item.split(":")[1]) for item in failed_files]
        outdf = self.meta_handle.read_df("global", read_metadata=False)

        tempmeta = outdf.reset_index(inplace=False)
        tempmeta = tempmeta.set_index(
            ["File Index", "Image Index"], drop=True, append=False, inplace=False
        )
        failed_fovs = tempmeta.loc[failed_file_idx]["fov"].unique().tolist()

        outdf = outdf.drop(failed_fovs)

        if self.t_range != None:
            outdf = outdf.reset_index(inplace=False)
            outdf["timepoints"] = outdf["timepoints"] - self.t_range[0]
            outdf = outdf.set_index(
                ["fov", "timepoints"], drop=True, append=False, inplace=False
            )

        self.meta_handle.write_df("global", outdf, metadata=self.metadata)

## Extract to hdf5 files

In this section, we will be extracting our image data. Currently this notebook only supports `.nd2` format; however <br>there are `.tiff` extractors in the TrenchRipper source files that are being added to `Master.ipynb` soon.

In the abstract, this step will take a single `.nd2` file and split it into a set of `.hdf5` files stored in <br>`headpath/hdf5`. Splitting the file up in this way will facilitate quick procesing in later steps. Each field of <br>view will be split into one or more `.hdf5` files, depending on the number of images per file requested (more on <br>this later). 

To keep track of which output files correspond to which FOVs, as well as to keep track of experiment metadata, the <br>extractor also outputs a `metadata.hdf5` file in the `headpath` folder. The data from this step is accessible in <br>that `metadata.hdf5` file under the `global` key. If you would like to look at this metadata, you may use the <br>`tr.utils.pandas_hdf5_handler` to read from this file. Later steps will add additional metadata under different <br>keys into the `metadata.hdf5` file.

#### Start Dask Workers

First, we start a `dask_controller` instance which will handle all of our parallel processing. The default parameters <br>here work well on O2. The critical arguments here are:

**walltime** : For a cluster, the length of time you will request each node for.

**local** : `True` if you want to perform computation locally. `False` if you want to perform it on a SLURM cluster.

**n_workers** : Number of nodes to request if on the cluster, or number of processes if computing locally.

**memory** : For a cluster, the amount of memory you will request each node for.

**working_directory** : For a cluster, the directory in which data will be spilled to disk. Usually set as a folder in <br>the `headpath`.

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="04:00:00",
    local=False,
    n_workers=40,
    memory="2GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

After running the above line, you will have a running Dask client. Run the line below and click the link to supervise <br>the computation being administered by the scheduler. 

Don't be alarmed if the screen starts mostly blank, it may take time for your workers to spin up. If you get a 404 <br>error on a cluster, it is likely that your ports are not being forwarded properly. If this occurs, please register <br>the issue on github.

In [ ]:
dask_controller.daskclient

In [ ]:
dask_controller.shutdown()

##### Perform Extraction

Now that we have our cluster scheduler spun up, it is time to convert files. This will be handled by the <br>`hdf5_extractor` object. This extractor will pull up each FOV and split it such that each derived `.hdf5` file <br>contains, at maximum, N timepoints of that FOV per file. The image data stored in these files takes the <br>form of `(N,Y,X)` arrays that are accessible using the desired channel name as a key. 

The arguments for this extractor are:

 - **nd2file** : The filepath to the `.nd2` file you intend to extract.
 
 - **headpath** : The folder in which processing is occuring. Should be the same for each step in the pipeline.

 - **tpts_per_file** : The maximum number of timepoints stored in each output `.hdf5` file. Typical values are between 25 <br>and 100.

 - **ignore_fovmetadata** : Used when `.nd2` data is corrupted and does not possess records for stage positions or <br>timepoints. Only set `False` if the extractor throws errors on metadata handling.

 - **nd2reader_override** : Overrides values in metadata recovered using the `nd2reader`. Currently set to <br>`{"z_levels":[],"z_coordinates":[]}` by default to correct a known issue where z coordinates are mistakenly <br>interpreted as a z stack. See the [nd2reader](https://rbnvrw.github.io/nd2reader/) documentation for more info.

In [ ]:
# hdf5_extractor = tr.marlin_extractor(hdf5inputpath, headpath, metaparsestr='metadata_{timepoint:d}.hdf5')

In [ ]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    nd2file,
    headpath,
    tpts_per_file=50,
    ignore_fovmetadata=False,
    nd2reader_override={"z_levels": [], "z_coordinates": []},
)

In [ ]:
# hdf5_extractor = tr.ndextract.tiff_extractor(
#     tiffpath,
#     headpath,
#     ["Phase","YFP"],tpts_per_file=50
# )

##### Extraction Parameters

Here, you may set the time interval you want to extract. Useful for cropping data to the period exhibiting the dynamics of interest.

Optionally take notes to add to the `metadata.hdf5` file. Notes may also be taken directly in this notebook.

In [ ]:
hdf5_extractor.inter_set_params()

##### Begin Extraction 

Running the following line will start the extraction process. This may be monitored by examining the `Dask Dashboard` <br> under the link displayed earlier. Once the computation is complete, move to the next line.

This step may take a long time, though it is possible to speed it up using additional workers.

In [ ]:
hdf5_extractor.extract(dask_controller)

##### Shutdown Dask

Once extraction is complete, it is likely that you will want to shutdown your `dask_controller` if you are on a <br>
cluster. This is because the specifications of the current `dask_controller` will not be optimal for later steps. <br>
To do this, run the following line and wait for it to complete. If it hangs, interrupt your kernel and re-run it. <br>
If this also fails to shutdown your workers, you will have to manually shut them down using `scancel` in a terminal.

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
dask_controller.shutdown()

## Kymographs

Now that you have extracted your data into a series of `.hdf5` files, we will now perform identification and cropping <br>of the individual trenches/growth channels present in the images. This algorithm assumes that your growth trenches <br>are vertically aligned and that they alternate in their orientation from top to bottom. See the example image for the <br>correct geometry:

![example_image](./resources/example_image.jpg)

The output of this step will be a set of `.hdf5` files stored in `headpath/kymograph`. The image data stored in these <br>files takes the form of `(K,T,Y,X)` arrays where K is the trench index, T is time, and Y,X are the crop dimensions. <br>These arrays are accessible using keys of the form `"[Image Channel]"`. For example, looking up phase channel <br>data of trenches in the topmost row of an image will require the key `"Phase"`

[ '/n/scratch3/users/d/de64/190917_20x_phase_gfp_segmentation002',
 '/n/scratch3/users/d/de64/190922_20x_phase_gfp_segmentation',
 '/n/scratch3/users/d/de64/190925_20x_phase_yfp_segmentation',
 '/n/scratch3/users/d/de64/ezrdm_training_sb7',
 '/n/scratch3/users/d/de64/mbm_training_sb7',
 '/n/scratch3/users/d/de64/Sb7_L35',
 '/n/scratch3/users/d/de64/MM_DVCvecto_TOP_1_9',
 '/n/scratch3/users/d/de64/Vibrio_2_1_TOP',
 '/n/scratch3/users/d/de64/Vibrio_A_B_VZRDM--04--RUN_80ms',
 '/n/scratch3/users/d/de64/RpoSOutliers_WT_hipQ_100X',
 '/n/scratch3/users/d/de64/Main_Experiment',
 '/n/scratch3/users/d/de64/bde17_gotime']

### Test Parameters



##### Initialize the interactive kymograph class

As a first step, initialize the `tr.interactive.kymograph_interactive` class that will be help us choose the <br>parameters we will use to generate kymographs. 

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
import numpy as np
import pandas as pd
import copy

In [ ]:
x_vals

In [ ]:
def get_grid_lookups(global_df, delta=10):
    first_tpt = global_df.loc[pd.IndexSlice[:, slice(0, 0)], :]

    x_vals = first_tpt["x"].values
    y_vals = first_tpt["y"].values

    x_dist = np.abs(np.subtract.outer(x_vals, x_vals))
    y_dist = np.abs(np.subtract.outer(y_vals, y_vals))

    close_x = x_dist < delta
    close_y = y_dist < delta

    x_groups = []
    for x_idx in range(close_x.shape[0]):
        x_groups.append(np.where(close_x[x_idx])[0])
    x_groups = [np.array(item) for item in set(list(tuple(arr) for arr in x_groups))]
    x_groups = sorted(x_groups, key=lambda x: x[0])
    x_lookup = {item: k for k, group in enumerate(x_groups) for item in group}

    y_groups = []
    for y_idx in range(close_y.shape[0]):
        y_groups.append(np.where(close_y[y_idx])[0])
    y_groups = [np.array(item) for item in set(list(tuple(arr) for arr in y_groups))]
    y_groups = sorted(y_groups, key=lambda x: x[0])
    y_lookup = {item: k for k, group in enumerate(y_groups) for item in group}

    return x_lookup, y_lookup


def get_grid_indices(global_df, delta=10):
    x_lookup, y_lookup = get_grid_lookups(global_df, delta=10)

    column_indices = [
        x_lookup[fov_idx]
        for fov_idx in global_df.index.get_level_values("fov").tolist()
    ]
    row_indices = [
        y_lookup[fov_idx]
        for fov_idx in global_df.index.get_level_values("fov").tolist()
    ]

    return column_indices, row_indices

In [ ]:
columns, rows = get_grid_indices(interactive_kymograph.metadf)
test = copy.deepcopy(interactive_kymograph.metadf)
test["Column"] = columns
test["Row"] = rows

In [ ]:
test

In [ ]:
test["column"] = column_indices

In [ ]:
test

In [ ]:
def infer_grid()

In [ ]:
viewer = tr.hdf5_viewer(headpath, persist_data=False)

##### Examine Images

Here you can manually inspect images before beginning parameter tuning.

In [ ]:
viewer.view(width=1200)

You will now want to select a few test FOVs to try out parameters on, the channel you want to detect trenches on, and <br>the time interval on which you will perform your processing.

The arguments for this step are:

- **seg_channel (string)** : The channel name that you would like to segment on.

- **invert (list)** : Whether or not you want to invert the image before detecting trenches. By default, it is assumed that <br>the trenches have a high pixel intensity relative to the background. This should be the case for Phase Contrast and <br>Fluorescence Imageing, but may not be the case for Brightfield Imaging, in which case you will want to invert the image.

- **fov_list (list)** : List of integers corresponding to the FOVs that you wish to make test kymographs of.

- **t_subsample_step (int)** : Step size to be used for subsampling input files in time, recommend that subsampling results in <br>between 5 and 10 timepoints for quick processing.

Hit the "Run Interact" button to lock in your parameters. The button will become transparent briefly and become solid again <br>when processing is complete. After that has occured, move on to the next step. 

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

The kymograph code begins by detecting the positions of trench rows in the image as follows:

1. Reducing each 2D image to a 1D signal along the y-axis by computing the qth percentile of the data along the x-axis
2. Smooth this signal using a median kernel
3. Normalize the signal by linearly scaling 0. and 1. to the minimum and maximum, respectively
4. Use a set threshold to determine the trench row poisitons

The arguments for this step are:

 - **y_percentile (int)** : Percentile to use for step 1.

 - **smoothing_kernel_y_dim_0 (int)** : Median kernel size to use for step 2.

 - **y_percentile_threshold (float)** : Threshold to use in step 4.

Running the following widget will display the smoothed 1-D signal for each of your timepoints. In addition, the threshold <br>value for each fov will be displayed as a red line.

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

Next, we will use the detected rows to perform cropping of the input image in the y-dimension:

1. Determine edges of trench rows based on threshold mask.
2. Filter out rows that are too small.
3. Use the remaining rows to compute the drift in y in each image.
4. Apply the drift to the initally detected rows to get rows in all timepoints.
5. Perform cropping using the "end" of the row as reference (the end referring to the part of the trench farthest from <br>the feeding channel).

Step 5 performs a simple algorithm to determine the orientation of each trench:

```
row_orientations = [] # A list of row orientations, starting from the topmost row
if the number of detected rows == 'Number of Rows': 
    row_orientations.append('Orientation')
elif the number of detected rows < 'Number of Rows':
    row_orientations.append('Orientation when < expected rows')
for row in rows:
    if row_orientations[-1] == downward:
        row_orientations.append(upward)
    elif row_orientations[-1] == upward:
        row_orientations.append(downward)
```

Additionally, if the device tranches face a single direction, alternation of row orientation may be turned off by setting the<br> `Alternate Orientation?` argument to False. The `Use Median Drift?` argument, when set to True, will use the<br> median drift in y across all FOVs for drift correction, instead of doing drift correction independently for all FOVs. <br>This can be useful if there are a large fraction of FOVs which are failing drift correction. Note that `Use Median Drift?` <br>sets this behavior for both y and x drift correction.

The arguments for this step are:

 - **y_min_edge_dist (int)** : Minimum row length necessary for detection (filters out small detected objects).

 - **padding_y (int)** : Padding to add to the end of trench row when cropping in the y-dimension.

 - **trench_len_y (int)** : Length from the end of each trench row to the feeding channel side of the crop.

 - **Number of Rows (int)** : The number of rows to expect in your image. For instance, two in the example image.
 
 - **Alternate Orientation? (bool)** : Whether or not to alternate the orientation of consecutive rows.

 - **Orientation (int)** : The orientation of the top-most row where 0 corresponds to a trench with a downward-oriented trench <br>opening and 1 corresponds to a trench with an upward-oriented trench opening.

 - **Orientation when < expected rows(int)** : The orientation of the top-most row when the number of detected rows is less than <br>expected. Useful if your trenches drift out of your image in some FOVs.
 
 - **Use Median Drift? (bool)** : Whether to use the median detected drift across all FOVs, instead of the drift detected in each FOV individually.

 - **images_per_row(int)** : How many images to output per row for this widget.

Running the following widget will display y-cropped images for each fov and timepoint.

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

Next, we will detect the positions of trenchs in the y-cropped images as follows:

1. Reducing each 2D image to a 1D signal along the x-axis by computing the qth percentile of the data along the y-axis.
2. Determine the signal background by smoothing this signal using a large median kernel.
3. Subtract the background signal.
4. Smooth the resultant signal using a median kernel.
5. Use an [otsu threhsold](https://imagej.net/Auto_Threshold#Otsu) to determine the trench midpoint poisitons.

After this, x-dimension drift correction of our detected midpoints will be performed as follows:

6. Begin at t=1
7. For $m \in \{midpoints(t)\}$ assign $n \in \{midpoints(t-1)\}$ to m if n is the closest midpoint to m at time $t-1$,<br>
points that are not the closest midpoint to any midpoints in m will not be mapped.
8. Compute the translation of each midpoint at time.
9. Take the average of this value as the x-dimension drift from time t-1 to t.

The arguments for this step are:

 - **t (int)** : Timepoint to examine the percentiles and threshold in.

 - **x_percentile (int)** : Percentile to use for step 1.

 - **background_kernel_x (int)** : Median kernel size to use for step 2.

 - **smoothing_kernel_x (int)** : Median kernel size to use for step 4.

 - **otsu_scaling (float)** : Scaling factor to apply to the threshold determined by Otsu's method.

Running the following widget will display the smoothed 1-D signal for each of your timepoints. In addition, the threshold <br>value for each fov will be displayed as a red line. In addition, it will display the detected midpoints for each of your timepoints. <br>If there is too much sparsity, or discontinuity, your drift correction will not be accurate.

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

Trench cropping simply uses the drift-corrected midpoints as a reference and crops out some fixed length around them <br>
to produce an output kymograph. **Note that the current implementation does not allow trench crops to overlap**. If your<br>
trench crops do overlap, the error will not be caught here, but will cause issues later in the pipeline. As such, try <br>
to crop your trenches as closely as possible. This issue will be fixed in a later update.

The arguments for this step are:

 - **trench_width_x (int)** : Trench width to use for cropping.

 - **trench_present_thr (float)** : Trenches that appear in less than this percent of FOVs will be eliminated from the dataset.<br>
If not removed, missing positions will be inferred from the image drift.

 - **Use Median Drift? (bool)** : Whether to use the median detected drift across all FOVs, instead of the drift detected in each FOV individually.


Running the following widget will display a random kymograph for each row in each fov and will also produce midpoint plots <br>showing retained midpoints

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

Run the following line to register and display the parameters you have selected for kymograph creation.

In [ ]:
interactive_kymograph.process_results()

If you are satisfied with the above parameters, run the following line to write these parameters to disk at `headpath/kymograph.par`<br>
This file will be used to perform kymograph creation in the next section.

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Start Dask Workers

Again, we start a `dask_controller` instance which will handle all of our parallel processing. The default parameters <br>here work well on O2 for kymograph creation. The critical arguments here are:

**walltime** : For a cluster, the length of time you will request each node for.

**local** : `True` if you want to perform computation locally. `False` if you want to perform it on a SLURM cluster.

**n_workers** : Number of nodes to request if on the cluster, or number of processes if computing locally.

**memory** : For a cluster, the amount of memory you will request each node for.

**working_directory** : For a cluster, the directory in which data will be spilled to disk. Usually set as a folder in <br>the `headpath`.

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="04:00:00",
    local=False,
    n_workers=50,
    memory="2GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

After running the above line, you will have a running Dask client. Run the line below and click the link to supervise <br>the computation being administered by the scheduler. 

Don't be alarmed if the screen starts mostly blank, it may take time for your workers to spin up. If you get a 404 <br>error on a cluster, it is likely that your ports are not being forwarded properly. If this occurs, please register <br>the issue on github.

In [ ]:
dask_controller.daskclient

##### Perform Kymograph Cropping

Now that we have our cluster scheduler spun up, we will extract kymographs using the parameters stored in `headpath/kymograph.par`. <br>
This will be handled by the `kymograph_cluster` object. This will detect trenches in all of the files present in `headpath/hdf5` that <br>
you created in the first step. It will then crop these trenches and place the crops in a series of `.hdf5` files in `headpath/kymograph`. <br>
These files will store image data in the form of `(K,T,Y,X)` arrays where K is the trench index, T is time and Y,X are the image dimensions <br>
of the crop.

The arguments for this step are:

 - **headpath** : The folder in which processing is occuring. Should be the same for each step in the pipeline.

 - **trenches_per_file** : The maximum number of trenches stored in each output `.hdf5` file. Typical values are between 25 <br>and 100.

 - **paramfile** : Set to true if you want to use parameters from `headpath/kymograph.par` Otherwise, you will have to specify <br>
 parameters as direct arguments to `kymograph_cluster`.

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, trenches_per_file=200, paramfile=True
)

##### Begin Kymograph Cropping 

Running the following line will start the cropping process. This may be monitored by examining the `Dask Dashboard` <br>
under the link displayed earlier. Once the computation is complete, move to the next line.

**Do not move on until all tasks are displayed as 'in memory' in Dask.**

In [ ]:
kymoclust.generate_kymographs(dask_controller)

In [ ]:
ff = tr.focus_filter(headpath)

In [ ]:
ff.choose_filter_channel_inter()

In [ ]:
ff.plot_histograms()

In [ ]:
ff.plot_focus_threshold_inter()

In [ ]:
ff.write_param_file()

##### Post-process Images

After the above step, kymographs will have been created for each `.hdf5` input file. They will now need to be reorganized <br>
into a new set of files such that each file has, at most, `trenches_per_file` trenches in each file.

**Do not move on until all tasks are displayed as 'in memory' in Dask.**

In [ ]:
kymoclust.post_process(dask_controller)

##### Check kymograph statistics

Run the next line to display some statistics from kymograph creation. The outputs are:

 - **fovs processed** : The number of FOVs successfully processed out of the total number of FOVs
 - **rows processed** : The number of rows of trenches processed out of the total number of rows
 - **trenches processed** : The number of trenches successfully processed
 - **row/fov** : The average number of rows successfully processed per FOV
 - **trenches/fov** : The average number of trenches successfully processed per FOV
 - **failed fovs** : A list of failed FOVs. Spot check these FOVs in the viewer to determine potential problems

In [ ]:
kymoclust.kymo_report()

##### Shutdown Dask

Once cropping is complete, it is likely that you will want to shutdown your `dask_controller` if you are on a <br>
cluster. This is because the specifications of the current `dask_controller` will not be optimal for later steps. <br>
To do this, run the following line and wait for it to complete. If it hangs, interrupt your kernel and re-run it. <br>
If this also fails to shutdown your workers, you will have to manually shut them down using `scancel` in a terminal.

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
dask_controller.shutdown()

## Fluorescence Segmentation

Now that you have copped your data into kymographs, we will now perform segmentation/cell detection <br>
on your kymographs. Currently, this pipeline only supports segmentation of fluorescence images; however, <br>
segmentation of transmitted light imaging techniques is in development.

The output of this step will be a set of `segmentation_[File #].hdf5` files stored in `headpath/fluorsegmentation`.<br>
The image data stored in these files takes the exact same form as the kymograph data, `(K,T,Y,X)` arrays <br>
where K is the trench index, T is time, and Y,X are the crop dimensions. These arrays are accessible using <br>
keys of the form `"[Trench Row Number]"`.

Since no metadata is generated by this step, it is possible to use another segmentation algorithm on the kymograph <br>
data. The output of segmentation must be split into `segmentation_[File #].hdf5` files, where `[File #]` agrees with the<br>
corresponding `kymograph_[File #].hdf5` file. Additionally, the `(K,T,Y,X)` arrays must be of the same shape as the <br>
kymograph arrays and accessible at the corresponding `"[Trench Row Number]"` key. These files must be placed into <br>
their own folder at `headpath/foldername`. This folder may then be used in later steps.

### Test Parameters

##### Initialize the interactive segmentation class

As a first step, initialize the `tr.fluo_segmentation_interactive` class that will be handling all steps of generating a segmentation. 

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
interactive_segmentation.choose_seg_channel_inter()

#### Import data

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**fov_idx (int)** :

**n_trenches (int)** :

**t_range (tuple)** :

**t_subsample_step (int)** :

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**cell_mask_method (str)** : Thresholding method, can be a local or global Otsu threshold.

**cell_otsu_scaling (float)** : Scaling factor applied to determined threshold.

**local_otsu_r (int)** : Radius of thresholding kernel used in the local otsu thresholding.

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="01:00:00",
    local=False,
    n_workers=50,
    memory="1GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath, paramfile=True)

In [ ]:
segment.dask_segment(dask_controller)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Region Properties (No Lineage)

Note this does not require a dask client

In [ ]:
from paulssonlab.deaton.trenchripper.trenchripper import pandas_hdf5_handler
from paulssonlab.deaton.trenchripper.trenchripper import dask_controller

import h5py
import os

import skimage as sk
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.delayed as delayed

from distributed.client import futures_of
from time import sleep

from matplotlib import pyplot as plt


class regionprops_extractor:
    def __init__(
        self,
        headpath,
        segmentationdir,
        intensity_channel_list=None,
        include_background=False,
        props=["centroid", "area", "mean_intensity"],
        unpack_dict={"centroid": ["centroid_y", "centroid_x"]},
    ):
        self.headpath = headpath
        self.intensity_channel_list = intensity_channel_list
        self.intensity_channel_dict = {
            channel: i for i, channel in enumerate(intensity_channel_list)
        }
        self.include_background = include_background
        self.kymographpath = headpath + "/kymograph"
        self.segmentationpath = headpath + "/" + segmentationdir
        self.metapath = self.kymographpath + "/metadata"
        self.analysispath = headpath + "/analysis"
        self.props = props
        self.unpack_dict = unpack_dict

    def get_file_regionprops(self, file_idx):
        segmentation_file = (
            self.segmentationpath + "/segmentation_" + str(file_idx) + ".hdf5"
        )
        kymograph_file = self.kymographpath + "/kymograph_" + str(file_idx) + ".hdf5"

        with h5py.File(segmentation_file, "r") as segfile:
            seg_arr = segfile["data"][:]
        if self.intensity_channel_list is not None:
            kymo_arr_list = []
            with h5py.File(kymograph_file, "r") as kymofile:
                for intensity_channel in self.intensity_channel_list:
                    kymo_arr_list.append(kymofile[intensity_channel][:])
        all_props_list = []
        for k in range(seg_arr.shape[0]):
            for t in range(seg_arr.shape[1]):
                labels = sk.measure.label(seg_arr[k, t])
                ## Measure regionprops of background pixels; will always be marked as the first object
                if self.include_background:
                    labels += 1
                if self.intensity_channel_list is not None:
                    for i, intensity_channel in enumerate(self.intensity_channel_list):
                        rps = sk.measure.regionprops(labels, kymo_arr_list[i][k, t])
                        props_list = []
                        for obj, rp in enumerate(rps):
                            props_entry = [file_idx, k, t, obj, intensity_channel]
                            for prop_key in self.props:
                                if prop_key in self.unpack_dict.keys():
                                    prop_split = self.unpack_dict[prop_key]
                                    prop_output = rp[prop_key]
                                    props_entry += [
                                        prop_output[i] for i in range(len(prop_split))
                                    ]
                                else:
                                    props_entry += [rp[prop_key]]
                            props_list.append(props_entry)
                        all_props_list += props_list

                #                         for prop_key in self.props:
                #                             props_list.append([file_idx, k, t, obj, intensity_channel])
                #                         props_list = [[file_idx, k, t, obj, intensity_channel]+[getattr(rp, prop_key) for prop_key in self.props] for obj,rp in enumerate(rps)]
                #                         all_props_list+=props_list
                else:
                    rps = sk.measure.regionprops(labels)
                    props_list = []
                    for obj, rp in enumerate(rps):
                        props_entry = [file_idx, k, t, obj]
                        for prop_key in self.props:
                            if prop_key in self.unpack_dict.keys():
                                prop_split = self.unpack_dict[prop_key]
                                prop_output = rp[prop_key]
                                props_entry += [
                                    prop_output[i] for i in range(len(prop_split))
                                ]
                            else:
                                props_entry += [rp[prop_key]]
                        props_list.append(props_entry)
                    all_props_list += props_list

        #                     props_list = [[file_idx, k, t, obj]+[getattr(rp, prop_key) for prop_key in self.props] for obj,rp in enumerate(rps)]
        #                     all_props_list+=props_list

        property_columns = [
            self.unpack_dict[prop] if prop in self.unpack_dict.keys() else [prop]
            for prop in self.props
        ]
        property_columns = [item for sublist in property_columns for item in sublist]

        if self.intensity_channel_list is not None:
            column_list = [
                "File Index",
                "File Trench Index",
                "timepoints",
                "Objectid",
                "Intensity Channel",
            ] + property_columns
            df_out = pd.DataFrame(all_props_list, columns=column_list).reset_index()
        else:
            column_list = [
                "File Index",
                "File Trench Index",
                "timepoints",
                "Objectid",
            ] + property_columns
            df_out = pd.DataFrame(all_props_list, columns=column_list).reset_index()

        file_idx = df_out.apply(
            lambda x: int(
                f"{x['File Index']:04}{x['File Trench Index']:04}{x['timepoints']:04}{x['Objectid']:02}{self.intensity_channel_dict[x['Intensity Channel']]:02}"
            ),
            axis=1,
        )

        df_out["File Parquet Index"] = [item for item in file_idx]
        df_out = df_out.set_index("File Parquet Index").sort_index()
        del df_out["index"]

        return df_out

    def analyze_all_files(self, dask_cont):
        df = dd.read_parquet(self.metapath)
        file_list = df["File Index"].unique().compute().tolist()
        #         kymo_meta = dd.read_parquet(self.metapath)
        #         file_list = kymo_meta["File Index"].unique().tolist()

        delayed_list = []
        for file_idx in file_list:
            df_delayed = delayed(self.get_file_regionprops)(file_idx)
            delayed_list.append(df_delayed.persist())

        ## filtering out non-failed dataframes ##
        all_delayed_futures = []
        for item in delayed_list:
            all_delayed_futures += futures_of(item)
        while any(future.status == "pending" for future in all_delayed_futures):
            sleep(0.1)

        good_delayed = []
        for item in delayed_list:
            if all([future.status == "finished" for future in futures_of(item)]):
                good_delayed.append(item)

        ## compiling output dataframe ##
        df_out = dd.from_delayed(good_delayed).persist()
        df_out["File Parquet Index"] = df_out.index
        df_out = df_out.set_index("File Parquet Index", drop=True, sorted=False)
        df_out = df_out.repartition(partition_size="25MB").persist()

        kymo_df = dd.read_parquet(self.metapath)
        kymo_df["File Merge Index"] = kymo_df["File Parquet Index"]
        kymo_df = kymo_df.set_index("File Merge Index", sorted=True)
        kymo_df = kymo_df.drop(
            ["File Index", "File Trench Index", "timepoints", "File Parquet Index"],
            axis=1,
        )

        df_out["File Merge Index"] = df_out.apply(
            lambda x: int(
                f'{x["File Index"]:04}{x["File Trench Index"]:04}{x["timepoints"]:04}'
            ),
            axis=1,
        )
        df_out = df_out.reset_index(drop=False)
        df_out = df_out.set_index("File Merge Index", sorted=True)

        df_out = df_out.join(kymo_df)
        df_out = df_out.set_index("File Parquet Index", sorted=True)

        dd.to_parquet(
            df_out,
            self.analysispath,
            engine="fastparquet",
            compression="gzip",
            write_metadata_file=True,
        )

    def export_all_data(self, n_workers=20, memory="8GB"):

        dask_cont = dask_controller(
            walltime="01:00:00",
            local=False,
            n_workers=n_workers,
            memory=memory,
            working_directory=self.headpath + "/dask",
        )
        dask_cont.startdask()
        dask_cont.displaydashboard()
        dask_cont.futures = {}

        try:
            self.analyze_all_files(dask_cont)
            dask_cont.shutdown()
        except:
            dask_cont.shutdown()
            raise

In [ ]:
def get_image_measurements(
    kymographpath, channels, file_idx, output_name, img_fn, *args, **kwargs
):

    df = dd.read_parquet(kymographpath + "/metadata")
    df = df.set_index("File Parquet Index", sorted=True)

    start_idx = int(str(file_idx) + "00000000")
    end_idx = int(str(file_idx) + "99999999")

    working_dfs = []

    proc_file_path = kymographpath + "/kymograph_" + str(file_idx) + ".hdf5"
    with h5py.File(proc_file_path, "r") as infile:
        working_filedf = df.loc[start_idx:end_idx].compute()
        trench_idx_list = working_filedf["File Trench Index"].unique().tolist()
        for trench_idx in trench_idx_list:
            trench_df = working_filedf[
                working_filedf["File Trench Index"] == trench_idx
            ]
            for channel in channels:
                kymo_arr = infile[channel][trench_idx]
                fn_out = [
                    img_fn(kymo_arr[i], *args, **kwargs)
                    for i in range(kymo_arr.shape[0])
                ]
                trench_df[channel + " " + output_name] = fn_out
            working_dfs.append(trench_df)

    out_df = pd.concat(working_dfs)
    return out_df


def get_all_image_measurements(
    headpath, output_path, channels, output_name, img_fn, *args, **kwargs
):
    kymographpath = headpath + "/kymograph"
    df = dd.read_parquet(kymographpath + "/metadata")

    file_list = df["File Index"].unique().compute().tolist()

    delayed_list = []
    for file_idx in file_list:
        df_delayed = delayed(get_image_measurements)(
            kymographpath, channels, file_idx, output_name, img_fn, *args, **kwargs
        )
        delayed_list.append(df_delayed.persist())

    ## filtering out non-failed dataframes ##
    all_delayed_futures = []
    for item in delayed_list:
        all_delayed_futures += futures_of(item)
    while any(future.status == "pending" for future in all_delayed_futures):
        sleep(0.1)

    good_delayed = []
    for item in delayed_list:
        if all([future.status == "finished" for future in futures_of(item)]):
            good_delayed.append(item)

    ## compiling output dataframe ##
    df_out = dd.from_delayed(good_delayed).persist()
    df_out["FOV Parquet Index"] = df_out.index
    df_out = df_out.set_index("FOV Parquet Index", drop=True, sorted=False)
    df_out = df_out.repartition(partition_size="25MB").persist()

    dd.to_parquet(
        df_out,
        output_path,
        engine="fastparquet",
        compression="gzip",
        write_metadata_file=True,
    )

In [ ]:
analyzer = regionprops_extractor(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/gfp",
    "fluorsegmentation",
    intensity_channel_list=["RFP-Penta", "GFP-Penta"],
    include_background=True,
)

In [ ]:
analyzer.export_all_data()

In [ ]:
kymo_df = dd.read_parquet(analyzer.metapath)
kymo_df["File Merge Index"] = kymo_df["File Parquet Index"]
kymo_df = kymo_df.set_index("File Merge Index", sorted=True)
kymo_df = kymo_df.drop(["File Index", "File Trench Index", "timepoints"], axis=1)

analysis_df = dd.read_parquet(analyzer.analysispath)
analysis_df["File Merge Index"] = analysis_df.apply(
    lambda x: int(
        f'{x["File Index"]:04}{x["File Trench Index"]:04}{x["timepoints"]:04}'
    ),
    axis=1,
)
analysis_df = analysis_df.set_index("File Merge Index", sorted=True)

merged_df = analysis_df.join(kymo_df)
merged_df = merged_df.set_index("File Parquet Index", sorted=True)

In [ ]:
merged_df

In [ ]:
import dask.dataframe as dd
import dask.delayed as delayed
from distributed.client import futures_of
import numpy as np
import pandas as pd
import h5py
import seaborn as sns
import scipy.signal
import skimage as sk
from time import sleep
from matplotlib import pyplot as plt

In [ ]:
region_props = pd.read_pickle(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/gfp/analysis.pkl"
)  # .loc[(slice(None), slice(None), list(range(10,20)), slice(None))]
region_props = region_props.reset_index()
region_props = region_props.set_index(
    ["trenchid", "timepoints", "Intensity Channel", "Objectid"], drop=True
)
region_props = region_props.sort_index()

#### Output

At this point you may want to use your output. The output of this step is a set of `.hdf5` files stored in <br>`headpath/kymograph`. The image data stored in these files takes the form of `(K,T,Y,X)` arrays <br>where K is the trench index, T is time, and Y,X are the crop dimensions.

These arrays are accessible using keys of the form `"[Trench Row Number]/[Image Channel]"`. <br>For example, looking up phase channel data of trenches in the topmost row of an image will require <br>the key `"0/Phase"` The metadata associated with these files is a large pandas dataframe relating <br>crops to original FOVs, accessible using the "kymograph" key on `headpath/metadata.hdf5`

To assist in accessing this file, you may use the `trenchripper.pandas_hdf5_handler` object to <br>interface with this file as follows:

In [ ]:
import dask.dataframe as dd
import dask.delayed as delayed
from distributed.client import futures_of
import numpy as np
import pandas as pd
import h5py
import scipy.signal
import skimage as sk
from time import sleep

In [ ]:
def get_image_measurements(
    kymographpath, channels, file_idx, output_name, img_fn, *args, **kwargs
):

    df = dd.read_parquet(kymographpath + "/metadata")

    working_dfs = []

    proc_file_path = kymographpath + "/kymograph_" + str(file_idx) + ".hdf5"
    with h5py.File(proc_file_path, "r") as infile:
        working_filedf = df[df["File Index"] == file_idx].compute()
        trench_idx_list = working_filedf["File Trench Index"].unique().tolist()
        for trench_idx in trench_idx_list:

            trench_df = working_filedf[
                working_filedf["File Trench Index"] == trench_idx
            ]

            for channel in channels:
                kymo_arr = infile[channel][trench_idx]
                fn_out = [
                    img_fn(kymo_arr[i], *args, **kwargs)
                    for i in range(kymo_arr.shape[0])
                ]
                trench_df[channel + " " + output_name] = fn_out
            working_dfs.append(trench_df)

    out_df = pd.concat(working_dfs)
    return out_df


def get_all_image_measurements(
    headpath, output_path, channels, output_name, img_fn, *args, **kwargs
):
    kymographpath = headpath + "/kymograph"
    df = dd.read_parquet(kymographpath + "/metadata")

    file_list = df["File Index"].unique().compute().tolist()

    delayed_list = []
    for file_idx in file_list:
        df_delayed = delayed(get_image_measurements)(
            kymographpath, channels, file_idx, output_name, img_fn, *args, **kwargs
        )
        delayed_list.append(df_delayed.persist())

    ## filtering out non-failed dataframes ##
    all_delayed_futures = []
    for item in delayed_list:
        all_delayed_futures += futures_of(item)
    while any(future.status == "pending" for future in all_delayed_futures):
        sleep(0.1)

    good_delayed = []
    for item in delayed_list:
        if all([future.status == "finished" for future in futures_of(item)]):
            good_delayed.append(item)

    ## compiling output dataframe ##
    df_out = dd.from_delayed(good_delayed).persist()
    df_out["FOV Parquet Index"] = df_out.index
    df_out = df_out.set_index("FOV Parquet Index", drop=True, sorted=False)
    df_out = df_out.repartition(partition_size="25MB").persist()

    dd.to_parquet(
        df_out,
        output_path,
        engine="fastparquet",
        compression="gzip",
        write_metadata_file=True,
    )

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/barcodes"

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="02:00:00",
    local=False,
    n_workers=30,
    memory="4GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
kymograph_metadata = dd.read_parquet(headpath + "/kymograph/metadata")

# kymograph_metadata = pd.read_parquet(headpath + "/kymograph/metadata")
# kymograph_metadata = kymograph_metadata[kymograph_metadata["fov"]>125]

In [ ]:
import numpy as np

tr.get_all_image_measurements(
    headpath,
    headpath + "/percentiles",
    ["RFP", "Cy5", "Cy7"],
    "98th Percentile",
    np.percentile,
    98,
)

In [ ]:
import h5py
import os

import skimage as sk
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.delayed as delayed

from distributed.client import futures_of
from time import sleep


def get_image_measurements(
    kymographpath, channels, file_idx, output_name, img_fn, *args, **kwargs
):

    df = dd.read_parquet(kymographpath + "/metadata")
    df = df.set_index("File Parquet Index", sorted=True)

    start_idx = int(str(file_idx) + "00000000")
    end_idx = int(str(file_idx) + "99999999")

    working_dfs = []

    proc_file_path = kymographpath + "/kymograph_" + str(file_idx) + ".hdf5"
    with h5py.File(proc_file_path, "r") as infile:
        working_filedf = df.loc[start_idx:end_idx].compute()
        trench_idx_list = working_filedf["File Trench Index"].unique().tolist()
        for trench_idx in trench_idx_list:
            trench_df = working_filedf[
                working_filedf["File Trench Index"] == trench_idx
            ]
            for channel in channels:
                kymo_arr = infile[channel][trench_idx]
                fn_out = [
                    img_fn(kymo_arr[i], *args, **kwargs)
                    for i in range(kymo_arr.shape[0])
                ]
                trench_df[channel + " " + output_name] = fn_out
            working_dfs.append(trench_df)

    out_df = pd.concat(working_dfs)
    return out_df


def get_all_image_measurements(
    headpath, output_path, channels, output_name, img_fn, *args, **kwargs
):
    kymographpath = headpath + "/kymograph"
    df = dd.read_parquet(kymographpath + "/metadata")

    file_list = df["File Index"].unique().compute().tolist()

    delayed_list = []
    for file_idx in file_list:
        df_delayed = delayed(get_image_measurements)(
            kymographpath, channels, file_idx, output_name, img_fn, *args, **kwargs
        )
        delayed_list.append(df_delayed.persist())

    ## filtering out non-failed dataframes ##
    all_delayed_futures = []
    for item in delayed_list:
        all_delayed_futures += futures_of(item)
    while any(future.status == "pending" for future in all_delayed_futures):
        sleep(0.1)

    good_delayed = []
    for item in delayed_list:
        if all([future.status == "finished" for future in futures_of(item)]):
            good_delayed.append(item)

    ## compiling output dataframe ##
    df_out = dd.from_delayed(good_delayed).persist()
    df_out["FOV Parquet Index"] = df_out.index
    df_out = df_out.set_index("FOV Parquet Index", drop=True, sorted=False)
    df_out = df_out.repartition(partition_size="25MB").persist()

    dd.to_parquet(
        df_out,
        output_path,
        engine="fastparquet",
        compression="gzip",
        write_metadata_file=True,
    )

In [ ]:
df = dd.read_parquet(headpath + "/kymograph/metadata")
df = df.set_index("File Parquet Index", sorted=True)

In [ ]:
df

In [ ]:
file_idx = 10
start_idx = int(str(file_idx) + "00000000")
end_idx = int(str(file_idx) + "99999999")

In [ ]:
df.loc[start_idx:end_idx].compute()

In [ ]:
get_all_image_measurements(
    headpath,
    headpath + "/percentiles",
    ["RFP", "Cy5", "Cy7"],
    "98th Percentile",
    np.percentile,
    98,
)

In [ ]:
import itertools
import numpy as np
import pandas as pd
import skimage as sk
import dask.array as da
import scipy.signal
from matplotlib import pyplot as plt
import scipy as sp
import sklearn as skl
import sklearn.mixture
import json
import copy

import dask.dataframe as dd

import ipywidgets as ipyw
from ipywidgets import (
    Checkbox,
    interact,
    interactive,
    fixed,
    interact_manual,
    FloatSlider,
    IntSlider,
    Dropdown,
    IntText,
    SelectMultiple,
    Select,
    IntRangeSlider,
    FloatRangeSlider,
)


def remove_bits_from_array(array, removed_bit_cycles=[]):
    out_array = copy.copy(array)
    out_array[removed_bit_cycles] = 0.0
    return out_array


def get_signal_sum(df, barcode_len=30, removed_bit_cycles=[], epsilon=0.1):
    trench_group = df.groupby(["trenchid"])
    barcodes = trench_group.apply(
        lambda x: np.array(
            x["RFP 98th Percentile"].tolist()
            + x["Cy5 98th Percentile"].tolist()
            + x["Cy7 98th Percentile"].tolist()
        ).astype(float)
    )

    short = barcodes.apply(lambda x: len(x) != barcode_len)
    for idx in np.where(short)[0]:
        barcodes[idx] = np.array([0.0 for i in range(barcode_len)])

    barcodes_arr = np.array(barcodes.to_list())
    barcodes_arr[:, removed_bit_cycles] = 0.0
    barcodes_arr_no_short = np.array(barcodes[~short].to_list())

    barcodes_median = np.median(barcodes_arr_no_short, axis=0) + epsilon

    barcodes_arr = barcodes_arr / barcodes_median[np.newaxis, :]
    barcodes_arr_no_short = barcodes_arr_no_short / barcodes_median[np.newaxis, :]

    signal_sum = np.sum(barcodes_arr, axis=1)
    signal_sum_no_short = np.sum(barcodes_arr_no_short, axis=1)

    signal_filter_thr = get_background_thr(
        signal_sum_no_short, get_background_dist_peak
    )

    return signal_sum, signal_filter_thr, barcodes_median, barcodes


def get_background_thr(values, background_fn, background_scaling=10.0):
    mu_n, std_n = background_fn(values)
    back_thr = mu_n + background_scaling * std_n
    return back_thr


def get_background_dist_peak(values):
    hist_range = (0, np.percentile(values, 90))
    hist_count, hist_vals = np.histogram(values, bins=100, range=hist_range)
    peaks = sp.signal.find_peaks(hist_count, distance=20)[0]
    mu_n = hist_vals[peaks[0]]
    lower_tail = values[values < mu_n]
    std_n = sp.stats.halfnorm.fit(-lower_tail)[1]
    return mu_n, std_n


def get_gmm_hard_assign(values):
    gmm = skl.mixture.GaussianMixture(n_components=2, n_init=10)
    gmm.fit(values.reshape(-1, 1))
    lower_mean_idx = np.argmin(gmm.means_)
    assign = gmm.predict(values.reshape(-1, 1))
    if lower_mean_idx == 1:
        assign = (-assign) + 1
    return assign


def barcode_to_FISH(
    barcodestr,
    cycleorder=[
        0,
        1,
        6,
        7,
        12,
        13,
        18,
        19,
        24,
        25,
        2,
        3,
        8,
        9,
        14,
        15,
        20,
        21,
        26,
        27,
        4,
        5,
        10,
        11,
        16,
        17,
        22,
        23,
        28,
        29,
    ],
):

    barcode = [bool(int(item)) for item in list(barcodestr)]
    FISH_barcode = np.array([barcode[i] for i in cycleorder])
    FISH_barcode = "".join(FISH_barcode.astype(int).astype(str))

    return FISH_barcode


class fish_analysis:
    def __init__(
        self,
        headpath,
        nanoporedfpath,
        nanoporejsonpath,
        subsample=5000,
        barcode_len=30,
        hamming_thr=0,
        channel_names=[
            "RFP 98th Percentile",
            "Cy5 98th Percentile",
            "Cy7 98th Percentile",
        ],
        cycleorder=[
            0,
            1,
            6,
            7,
            12,
            13,
            18,
            19,
            24,
            25,
            2,
            3,
            8,
            9,
            14,
            15,
            20,
            21,
            26,
            27,
            4,
            5,
            10,
            11,
            16,
            17,
            22,
            23,
            28,
            29,
        ],
    ):
        self.headpath = headpath
        self.nanoporedfpath = nanoporedfpath
        self.nanoporejsonpath = nanoporejsonpath
        self.subsample = subsample
        self.barcode_len = barcode_len
        self.hamming_thr = hamming_thr
        self.channel_names = channel_names
        self.cycleorder = cycleorder
        self.percentilepath = headpath + "/percentiles"

        with open(nanoporejsonpath) as in_file:
            self.nanopore_metadata = json.load(in_file)

        self.removed_bits = self.nanopore_metadata["removed_bits"]
        self.removed_bit_cycles = [cycleorder[bit] for bit in self.removed_bits]

        kymograph_metadata = dd.read_parquet(self.percentilepath)
        kymograph_metadata = kymograph_metadata.set_index(
            "trenchid", drop=True, sorted=True
        )

        self.trenchids = list(
            kymograph_metadata.index.compute().get_level_values(0).unique()
        )
        if (self.subsample != None) and (self.subsample < len(self.trenchids)):
            selected_trenchids = sorted(
                list(np.random.choice(self.trenchids, (self.subsample,), replace=False))
            )
            kymograph_metadata = kymograph_metadata.loc[selected_trenchids]

        self.kymograph_metadata_subsample = (
            kymograph_metadata.compute()
            .reset_index()
            .set_index(["trenchid", "timepoints"], drop=True)
        )
        (
            self.signal_sum,
            self.signal_filter_thr,
            self.barcodes_median,
            self.barcodes,
        ) = get_signal_sum(
            self.kymograph_metadata_subsample,
            barcode_len=self.barcode_len,
            removed_bit_cycles=self.removed_bit_cycles,
        )

    def plot_signal_threshold(self, signal_filter_thr, figsize=(12, 8)):
        self.signal_filter_thr = signal_filter_thr
        high_signal_mask = self.signal_sum > self.signal_filter_thr
        high_signal_barcodes_series = self.barcodes[high_signal_mask]
        self.high_signal_barcodes = np.array(
            [item for item in high_signal_barcodes_series]
        )

        max_v = np.percentile(self.signal_sum, 99)

        fig = plt.figure(figsize=figsize)
        plt.hist(
            self.signal_sum[high_signal_mask],
            bins=100,
            range=(0, max_v),
            color="steelblue",
        )
        plt.hist(
            self.signal_sum[~high_signal_mask], bins=100, range=(0, max_v), color="grey"
        )
        plt.title("Sum of all barcode signal", fontsize=20)
        plt.xlabel("Summed Intensity", fontsize=20)
        plt.ylabel("Number of Trenches", fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        plt.show()

    def plot_signal_threshold_inter(self):
        signal_list_int = interactive(
            self.plot_signal_threshold,
            {"manual": True},
            signal_filter_thr=FloatSlider(
                value=self.signal_filter_thr,
                description="Signal Threshold:",
                min=0.0,
                max=100.0,
                step=0.5,
                disabled=False,
            ),
            figsize=fixed((12, 8)),
        )

        display(signal_list_int)

    def get_bit_thresholds(self):
        bit_threshold_list = []
        omit_bit_list = []
        for i in range(self.barcode_len):
            values = fish_test.high_signal_barcodes[:, i]
            assign = get_gmm_hard_assign(values).astype(bool)
            threshold = (np.min(values[assign]) + np.max(values[~assign])) / 2
            bit_threshold_list.append(threshold)
            omit_bit_list.append(False)
        self.bit_threshold_list = bit_threshold_list

    def plot_bit_threshold(self, idx, bit_filter_thr, figsize=(12, 8)):
        self.bit_threshold_list[idx] = bit_filter_thr

        high_signal_mask = (
            self.high_signal_barcodes[:, idx] > self.bit_threshold_list[idx]
        )
        on_arr = self.high_signal_barcodes[:, idx][high_signal_mask]
        off_arr = self.high_signal_barcodes[:, idx][~high_signal_mask]

        max_val = np.percentile(self.high_signal_barcodes[:, idx].flatten(), 99)
        bins = np.linspace(0, max_val, num=50)
        on_frq, on_edges = np.histogram(on_arr, bins)
        off_frq, off_edges = np.histogram(off_arr, bins)

        plt.bar(
            off_edges[:-1],
            off_frq,
            width=np.diff(off_edges),
            align="edge",
            color="steelblue",
        )
        plt.bar(
            on_edges[:-1], on_frq, width=np.diff(on_edges), align="edge", color="grey"
        )
        plt.title("Barcode Signal", fontsize=20)
        plt.xlabel("Intensity", fontsize=20)
        plt.ylabel("Number of Trenches", fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        plt.show()

    def plot_bit_threshold_inter(self):
        timepoints = (
            self.kymograph_metadata_subsample.index.get_level_values("timepoints")
            .unique()
            .tolist()
        )

        idx = 0
        channel_children = []
        for j, channel_name in enumerate(self.channel_names):

            tpt_children = []
            for i, tpt in enumerate(timepoints):
                tpt_children.append(
                    interactive(
                        self.plot_bit_threshold,
                        idx=fixed(idx),
                        bit_filter_thr=IntText(
                            value=self.bit_threshold_list[idx],
                            description="Bit " + str(idx),
                            min=0,
                            max=10000000,
                            disabled=False,
                        ),
                        figsize=fixed((12, 8)),
                    )
                )
                idx += 1
            tpt_tab = ipyw.Tab()
            tpt_tab.children = tpt_children
            for i in range(len(tpt_children)):
                tpt_tab.set_title(i, "Cycle " + str(i))

            channel_children.append(tpt_tab)

        channel_tab = ipyw.Tab()
        channel_tab.children = channel_children
        for i in range(len(channel_children)):
            channel_tab.set_title(i, self.channel_names[i])

        display(channel_tab)

    def get_barcode_df(self, epsilon=0.1):
        self.kymograph_metadata = dd.read_parquet(self.percentilepath)
        trench_group = self.kymograph_metadata.groupby(["trenchid"])
        barcodes = trench_group.apply(
            lambda x: np.array(
                list(
                    itertools.chain.from_iterable(
                        [x[channel].tolist() for channel in self.channel_names]
                    )
                )
            ).astype(float)
        ).compute()
        short = barcodes.apply(lambda x: len(x) != self.barcode_len)
        barcodes[short] = [
            np.array([0.0 for i in range(self.barcode_len)]) for i in range(len(short))
        ]

        barcodes = barcodes.apply(
            remove_bits_from_array, removed_bit_cycles=self.removed_bit_cycles
        )
        signal_sum = barcodes.apply(lambda x: np.sum(x / self.barcodes_median))

        high_signal_mask = signal_sum > self.signal_filter_thr
        high_signal_barcodes_series = barcodes[high_signal_mask]

        assign_arr = np.array(
            high_signal_barcodes_series.apply(
                lambda x: x > np.array(self.bit_threshold_list)
            ).to_list()
        ).T
        assign_arr[self.removed_bit_cycles] = False

        assign_strs = np.apply_along_axis(
            lambda x: "".join(x.astype(int).astype(str)), 0, assign_arr
        )
        barcode_df = pd.DataFrame(
            high_signal_barcodes_series, columns=["Barcode Signal"]
        )
        barcode_df["Barcode"] = assign_strs
        self.barcode_df = barcode_df.reset_index(drop=False)

    def get_nanopore_df(self):
        nanopore_df = pd.read_csv(self.nanoporedfpath, delimiter="\t", index_col=0)

        nanopore_lookup = {}
        for _, row in nanopore_df.iterrows():
            nanopore_lookup[
                barcode_to_FISH(row["barcode"], cycleorder=self.cycleorder)
            ] = row
        del nanopore_df
        self.nanopore_lookup_df = pd.DataFrame(nanopore_lookup).T
        del nanopore_lookup

    def get_merged_df(self):
        mergeddf = []

        if self.hamming_thr == 0:
            for _, row in self.barcode_df.iterrows():
                try:
                    nanopore_row = self.nanopore_lookup_df.loc[row["Barcode"]]
                    entry = pd.concat([row, nanopore_row])
                    mergeddf.append(entry)
                except:
                    pass

        else:
            nanopore_idx = np.array(
                [
                    np.array(list(item)).astype(bool)
                    for item in self.nanopore_lookup_df.index.tolist()
                ]
            ).astype(bool)
            nanopore_idx = da.from_array(nanopore_idx, chunks=(20000, 30))

            queries = np.array(
                [
                    np.array(list(item)).astype(bool)
                    for item in self.barcode_df["Barcode"].tolist()
                ]
            ).astype(bool)
            queries = da.from_array(queries, chunks=(20000, 30))

            match = (nanopore_idx.astype("uint8") @ queries.T.astype("uint8")) + (
                (~nanopore_idx).astype("uint8") @ (~queries).T.astype("uint8")
            )
            hamming_dist = self.barcode_len - match
            closest_match_thr = da.min(hamming_dist, axis=0) <= self.hamming_thr
            closest_match = da.argmin(hamming_dist, axis=0)
            closest_match[~closest_match_thr] = -1
            closest_match = closest_match.compute()

            filtered_lookup = {
                query_idx: target_idx
                for query_idx, target_idx in enumerate(closest_match)
                if target_idx != -1
            }

            mergeddf = []
            for i, row in self.barcode_df.iterrows():
                try:
                    nanopore_row = self.nanopore_lookup_df.iloc[filtered_lookup[i]]
                    entry = pd.concat([row, nanopore_row])
                    mergeddf.append(entry)
                except:
                    pass

        del self.nanopore_lookup_df
        self.mergeddf = pd.DataFrame(mergeddf)
        del mergeddf

    def output_barcode_df(self):
        self.get_barcode_df()
        self.get_nanopore_df()
        self.get_merged_df()

        ttl_trenches = len(self.kymograph_metadata["trenchid"].unique().compute())
        ttl_trenches_w_cells = len(self.barcode_df)

        output_handle = tr.pandas_hdf5_handler(self.headpath + "/metadata.hdf5")
        output_handle.write_df(
            "barcodes",
            self.mergeddf,
            metadata={
                "Total Trenches": ttl_trenches,
                "Total Trenches With Cells": ttl_trenches_w_cells,
                "Removed Bits": self.nanopore_metadata["removed_bits"],
                "Removed Bit Cycles": self.removed_bit_cycles,
                "Cycle Order": self.cycleorder,
                "Barcode Length": self.barcode_len,
                "Hamming Threshold": self.hamming_thr,
            },
        )

        del self.mergeddf

        # nanopore_df = pd.read_csv("../2020-11-07_lDE11/lDE11_final_df.tsv",delimiter="\t")
        # nanopore_lookup = {}
        # for _,row in nanopore_df.iterrows():
        #     nanopore_lookup[barcode_to_FISH(row["barcode"],cycleorder=fish_test.cycleorder)] = row["dark_gfp"]

        ## application specific?
        # barcode_df["Predicted Dark GFP"] = barcode_df.apply(lambda x: lDE11_lookup[x["Barcode"]] if x["Barcode"] in lDE11_lookup.keys() else "Unknown",axis=1)

In [ ]:
nanopore_df = pd.read_csv("./lDE14_final_df.tsv", delimiter="\t", index_col=0)

In [ ]:
with open("./lDE14_final_df.json") as in_file:
    nanopore_metadata = json.load(in_file)

In [ ]:
nanopore_metadata

In [ ]:
perc_one_away = np.sum(nanopore_df["Closest Hamming Distance"] == 1) / len(
    nanopore_df["Closest Hamming Distance"]
)

In [ ]:
perc_one_away

In [ ]:
fish_test = tr.fish_analysis(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/barcodes",
    "./lDE14_final_df.tsv",
    "./lDE14_final_df.json",
    hamming_thr=0,
)

In [ ]:
fish_test.plot_signal_threshold_inter()

In [ ]:
fish_test.get_bit_thresholds()

In [ ]:
fish_test.bit_threshold_list = [
    800,
    800,
    1000,
    1200,
    1200,
    1200,
    1500,
    1500,
    1200,
    1500,
    2500,
    4000,
    4000,
    3000,
    3500,
    3000,
    3500,
    3500,
    2500,
    5000,
    400,
    400,
    400,
    300,
    500,
    400,
    400,
    500,
    400,
    400,
]

In [ ]:
fish_test.plot_bit_threshold_inter()

In [ ]:
fish_test.output_barcode_df()

In [ ]:
meta_handle = tr.pandas_hdf5_handler(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/barcodes/metadata.hdf5"
)
barcode_df = meta_handle.read_df("barcodes", read_metadata=True)

In [ ]:
true_barcodes = np.array(
    barcode_df["barcode"].apply(lambda x: list(barcode_to_FISH(x))).tolist()
).astype("uint8")

In [ ]:
read_barcodes = np.array(
    [list(item) for item in barcode_df["Barcode"].tolist()]
).astype("uint8")

In [ ]:
true_barcodes.shape

In [ ]:
read_barcodes.shape

In [ ]:
bit_error = (
    np.sum(np.logical_xor(true_barcodes, read_barcodes), axis=0)
    / true_barcodes.shape[0]
)

In [ ]:
plt.bar(range(30), bit_error)
plt.show()

In [ ]:
ttl_true = np.sum([item == True for item in barcode_df["dark_gfp"].tolist()])
ttl_false = np.sum([item == False for item in barcode_df["dark_gfp"].tolist()])
ttl_none = np.sum([item == "Unknown" for item in barcode_df["dark_gfp"].tolist()])
ttl_called = ttl_true + ttl_false
ttl_trenches = barcode_df.metadata["Total Trenches"]
ttl_trenches_w_cells = barcode_df.metadata["Total Trenches With Cells"]
percent_called = ttl_called / ttl_trenches
percent_called_w_cells = ttl_called / ttl_trenches_w_cells

In [ ]:
percent_called

In [ ]:
percent_called_w_cells

In [ ]:
analysis_df = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/gfp/analysis"
)

In [ ]:
mchy_df = analysis_df[analysis_df["Intensity Channel"] == "RFP-Penta"]
mchy_groupby = mchy_df.groupby(["trenchid", "timepoints"])

gfp_df = analysis_df[analysis_df["Intensity Channel"] == "GFP-Penta"]
gfp_groupby = gfp_df.groupby(["trenchid", "timepoints"])

gfp_intensity_wo_bkd = (
    gfp_groupby.apply(
        lambda x: (
            x["mean_intensity"] - x[x["Objectid"] == 0]["mean_intensity"].iloc[0]
        ).to_dict(),
        meta=("mean_intensity", float),
    )
    .reset_index(drop=True)
    .compute()
    .to_list()
)
gfp_intensity_wo_bkd = {k: v for d in gfp_intensity_wo_bkd for k, v in d.items()}
gfp_intensity_wo_bkd = pd.DataFrame.from_dict(
    gfp_intensity_wo_bkd, orient="index", columns=["mean_intensity_wo_bkd"]
)
gfp_df = gfp_df.join(gfp_intensity_wo_bkd).persist()
del gfp_intensity_wo_bkd

mchy_intensity_wo_bkd = (
    mchy_groupby.apply(
        lambda x: (
            x["mean_intensity"] - x[x["Objectid"] == 0]["mean_intensity"].iloc[0]
        ).to_dict(),
        meta=("mean_intensity", float),
    )
    .reset_index(drop=True)
    .compute()
    .to_list()
)
mchy_intensity_wo_bkd = {k: v for d in mchy_intensity_wo_bkd for k, v in d.items()}
mchy_intensity_wo_bkd = pd.DataFrame.from_dict(
    mchy_intensity_wo_bkd, orient="index", columns=["mean_intensity_wo_bkd"]
)
mchy_df = mchy_df.join(mchy_intensity_wo_bkd).persist()
del mchy_intensity_wo_bkd

In [ ]:
gfp_df_nobkd = gfp_df[gfp_df["Objectid"] != 0]
gfp_df_nobkd["Object Parquet Index"] = gfp_df_nobkd.apply(
    lambda x: int(
        f"{x['File Index']:04}{x['File Trench Index']:04}{x['timepoints']:04}{x['Objectid']:02}"
    ),
    axis=1,
)
gfp_df_nobkd = gfp_df_nobkd.set_index("Object Parquet Index")

mchy_df_nobkd = mchy_df[mchy_df["Objectid"] != 0]
mchy_df_nobkd["Object Parquet Index"] = mchy_df_nobkd.apply(
    lambda x: int(
        f"{x['File Index']:04}{x['File Trench Index']:04}{x['timepoints']:04}{x['Objectid']:02}"
    ),
    axis=1,
)
mchy_df_nobkd = mchy_df_nobkd.set_index("Object Parquet Index")

ratio_series = (
    gfp_df_nobkd["mean_intensity_wo_bkd"] / mchy_df_nobkd["mean_intensity_wo_bkd"]
)
gfp_df_nobkd["gfp/mchy Ratio"] = ratio_series

In [ ]:
threshold = 2.0

trenchid_groupby = gfp_df_nobkd.groupby("trenchid")
median_ratio = trenchid_groupby["gfp/mchy Ratio"].apply(np.median).compute()
median_ratio = median_ratio.sort_index()
dark_gfp = median_ratio < threshold

In [ ]:
perc_gfp = 1.0 - (np.sum(dark_gfp) / len(median_ratio))

In [ ]:
print(perc_gfp)

In [ ]:
analysis_df.loc[:10].compute()

In [ ]:
plt.hist(
    median_ratio[median_ratio < threshold],
    range=(0, 20),
    bins=50,
    color="grey",
    label="Measured Dark GFP",
    density=True,
)
plt.hist(
    median_ratio[median_ratio > threshold],
    range=(0, 20),
    bins=50,
    color="green",
    label="Measured GFP",
    density=True,
)
plt.xlabel("Mean Intensity Ratio", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
# plt.savefig("./2021-03-10_lDE14_figure_2.png",dpi=300,bbox_inches="tight")
plt.show()

In [ ]:
gfp_kymo_df = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/gfp/kymograph/metadata"
)
barcode_kymo_df = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2021-01-28_lDE14/barcodes/kymograph/metadata"
)

max_gfp_tpt = gfp_kymo_df.loc[:1000]["timepoints"].max().compute()
min_barcode_tpt = barcode_kymo_df.loc[:1000]["timepoints"].min().compute()

last_gfp_tpt_df = gfp_kymo_df[gfp_kymo_df["timepoints"] == max_gfp_tpt].compute()
first_barcode_tpt_df = barcode_kymo_df[
    barcode_kymo_df["timepoints"] == min_barcode_tpt
].compute()

In [ ]:
len(last_gfp_tpt_df)

In [ ]:
len(first_barcode_tpt_df)

Register DFs

In [ ]:
def get_trenchid_map(kymodf1, kymodf2):
    trenchid_map = {}

    fovset1 = set(kymodf1["fov"].unique().tolist())
    fovset2 = set(kymodf2["fov"].unique().tolist())
    fov_intersection = fovset1.intersection(fovset2)

    for fov in fov_intersection:
        df1_chunk = kymodf1[kymodf1["fov"] == fov]
        df2_chunk = kymodf2[kymodf2["fov"] == fov]

        df1_xy = df1_chunk[["y (local)", "x (local)"]].values
        df2_xy = df2_chunk[["y (local)", "x (local)"]].values

        ymat = np.subtract.outer(df1_xy[:, 0], df2_xy[:, 0])
        xmat = np.subtract.outer(df1_xy[:, 1], df2_xy[:, 1])
        distmat = (ymat**2 + xmat**2) ** (1 / 2)

        # ensuring map is one-to-one
        mapping = np.argmin(distmat, axis=1)
        invmapping = np.argmin(distmat, axis=0)
        mapping = {
            idx: map_idx
            for idx, map_idx in enumerate(mapping)
            if invmapping[map_idx] == idx
        }

        df1_trenchids = df1_chunk["trenchid"].tolist()
        df2_trenchids = df2_chunk["trenchid"].tolist()

        trenchid_map.update(
            {
                trenchid: df2_trenchids[mapping[i]]
                for i, trenchid in enumerate(df1_trenchids)
                if i in mapping.keys()
            }
        )

    return trenchid_map

In [ ]:
trenchid_map = get_trenchid_map(first_barcode_tpt_df, last_gfp_tpt_df)

In [ ]:
len(trenchid_map)

In [ ]:
def map_Series(x, series, trenchid_map, dtype=str):
    if x["trenchid"] in trenchid_map.keys():
        if trenchid_map[x["trenchid"]] in series.index:
            return series.loc[trenchid_map[x["trenchid"]]]
        else:
            return "Unknown"
    else:
        return "Unknown"


barcode_df["Measured Dark GFP"] = barcode_df.apply(
    map_Series, axis=1, args=(dark_gfp, trenchid_map)
)
barcode_df["Measured GFP Ratio"] = barcode_df.apply(
    map_Series, axis=1, args=(median_ratio, trenchid_map)
)

In [ ]:
called_df = barcode_df[barcode_df["Measured Dark GFP"] != "Unknown"]

In [ ]:
ttl_correct = np.sum(called_df["dark_gfp"] == called_df["Measured Dark GFP"])
ttl_called = len(called_df)
print("Error Rate:" + str(1.0 - ttl_correct / ttl_called))

In [ ]:
plt.hist(
    called_df[called_df["Measured Dark GFP"] == True]["Measured GFP Ratio"],
    range=(0, 20),
    bins=50,
    color="grey",
    alpha=0.7,
    label="Predicted Dark GFP",
    density=True,
)
plt.hist(
    called_df[called_df["Measured Dark GFP"] == False]["Measured GFP Ratio"],
    range=(0, 20),
    bins=50,
    color="green",
    alpha=0.7,
    label="Predicted GFP",
    density=True,
)
plt.xlabel("Median Intensity", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
# plt.savefig("./2020-11-07_lDE11_figure_1.png",dpi=300,bbox_inches="tight")
plt.show()

In [ ]:
median_gfp_df = called_df.groupby("Barcode").apply(
    lambda x: x["Measured GFP Ratio"].median()
)

In [ ]:
plt.hist(
    median_gfp_df[median_gfp_df < threshold],
    range=(0, 20),
    bins=50,
    color="grey",
    label="Measured Dark GFP",
)
plt.hist(
    median_gfp_df[median_gfp_df > threshold],
    range=(0, 20),
    bins=50,
    color="green",
    label="Measured GFP",
)
plt.xlabel("Mean Intensity Ratio", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
# plt.savefig("./2020-10-10_lDE11_figure_2.png",dpi=300,bbox_inches="tight")
plt.show()

In [ ]:
called_df_barcodes = called_df.set_index(["Barcode"]).sort_index()
called_df_barcodes["Measured Median GFP"] = median_gfp_df
called_df_barcodes.reset_index(drop=False)
called_df_barcodes = called_df_barcodes.groupby("Barcode").apply(lambda x: x.iloc[0])

In [ ]:
called_df_barcodes

In [ ]:
ttl_correct = np.sum(
    called_df_barcodes["dark_gfp"] == (called_df_barcodes["Measured Median GFP"] < 2.0)
)
ttl_called = len(called_df_barcodes)
print("Percent Correct:" + str(ttl_correct / ttl_called))

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.hist(
    called_df_barcodes[called_df_barcodes["Predicted Dark GFP"] == True][
        "Measured Median GFP"
    ],
    range=(0, 3),
    bins=50,
    color="grey",
    alpha=0.7,
    label="Predicted Dark GFP",
)
plt.hist(
    called_df_barcodes[called_df_barcodes["Predicted Dark GFP"] == False][
        "Measured Median GFP"
    ],
    range=(0, 3),
    bins=50,
    color="green",
    alpha=0.7,
    label="Predicted GFP",
)
plt.xlabel("Median Intensity Ratio", fontsize=20)
plt.ylabel("# Strains", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
plt.tight_layout()
plt.savefig("./2020-11-07_lDE11_figure_1.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
called_df_barcodes

#### Get Confusion Matrix

In [ ]:
def get_confusion_mat(df):
    TP = np.sum((df["dark_gfp"] == False) & (df["Measured Dark GFP"] == False))
    TN = np.sum((df["dark_gfp"] == True) & (df["Measured Dark GFP"] == True))
    FP = np.sum((df["dark_gfp"] == False) & (df["Measured Dark GFP"] == True))
    FN = np.sum((df["dark_gfp"] == True) & (df["Measured Dark GFP"] == False))

    error = (FP + FN) / (TP + TN + FP + FN)
    FP_error = FP / (TP + TN + FP + FN)
    FN_error = FN / (TP + TN + FP + FN)

    return error, FP_error, FN_error

In [ ]:
error, FP_error, FN_error = get_confusion_mat(called_df_barcodes)
print("Error: " + str(error))
print("FP error: " + str(FP_error))
print("FN error: " + str(FN_error))

In [ ]:
called_df_barcodes

In [ ]:
import seaborn as sns

sns.set()

hamming_filters = list(range(1, 5))
hamming_n_barcodes = []
hamming_errors = []
for i in hamming_filters:
    filtered_df = called_df_barcodes[
        called_df_barcodes["Closest Hamming Distance"] >= i
    ]
    n_barcode = len(filtered_df)
    error, FP_error, FN_error = get_confusion_mat(filtered_df)
    error = np.round(100 * error, decimals=2)
    hamming_errors.append(error)
    hamming_n_barcodes.append(n_barcode)

sns.lineplot(hamming_filters, hamming_errors, linewidth=5)
plt.xticks(fontsize=20)
plt.yticks(
    fontsize=20,
)
plt.show()

sns.lineplot(hamming_filters, hamming_n_barcodes, linewidth=5)
plt.xticks(fontsize=20)
plt.yticks(
    fontsize=20,
)
plt.show()

In [ ]:
def bootstrap_subsample(df, value, subsample_n, bootstrap_n=500):
    bootstrap_errors = []
    for i in range(bootstrap_n):
        sub_df = df.sample(n=subsample_n)
        error, FP_error, FN_error = get_confusion_mat(sub_df)
        error = np.round(100 * error, decimals=2)
        bootstrap_errors.append(error)
    percentile = sp.stats.percentileofscore(bootstrap_errors, value)

    return percentile

In [ ]:
bootstrap_subsample(
    called_df_barcodes, hamming_errors[-1], hamming_n_barcodes[-1], bootstrap_n=1000
)

In [ ]:
def get_gmm_params(values):
    gmm = skl.mixture.GaussianMixture(n_components=2, n_init=10)
    gmm.fit(values.reshape(-1, 1))
    #     probs = gmm.predict_proba(values.reshape(-1,1))
    return gmm.means_[:, 0], ((gmm.covariances_) ** (1 / 2))[:, 0, 0]

In [ ]:
means, stds = get_gmm_params(called_df_barcodes["Measured Median GFP"].values)

In [ ]:
upper_bound = means + stds * 0.1
lower_bound = means - stds * 0.1

In [ ]:
upper_bound[0]

In [ ]:
valid_dark = (called_df_barcodes["Measured Median GFP"] < upper_bound[0]) & (
    called_df_barcodes["Measured Median GFP"] > lower_bound[0]
)
valid_gfp = (called_df_barcodes["Measured Median GFP"] < upper_bound[1]) & (
    called_df_barcodes["Measured Median GFP"] > lower_bound[1]
)
valid = valid_dark | valid_gfp

In [ ]:
len(called_df_barcodes[valid])

In [ ]:
import seaborn as sns

sns.set()

n_std = np.linspace(0, 5, 20)
n_barcodes = []
errors = []
FP_errors = []
FN_errors = []
for i in n_std:
    upper_bound = means + stds * i
    lower_bound = means - stds * i

    #     valid_dark = (called_df_barcodes["Measured Median GFP"] < upper_bound[0]) &\
    #     (called_df_barcodes["Measured Median GFP"] > lower_bound[0])
    #     valid_gfp = (called_df_barcodes["Measured Median GFP"] < upper_bound[1]) &\
    #     (called_df_barcodes["Measured Median GFP"] > lower_bound[1])
    #     valid = valid_dark|valid_gfp
    valid_dark = called_df_barcodes["Measured Median GFP"] < upper_bound[0]
    valid_gfp = called_df_barcodes["Measured Median GFP"] > lower_bound[1]
    valid = valid_dark | valid_gfp

    filtered_df = called_df_barcodes[valid]
    n_barcode = len(filtered_df)
    error, FP_error, FN_error = get_confusion_mat(filtered_df)
    error = np.round(100 * error, decimals=2)
    FP_error = np.round(100 * FP_error, decimals=2)
    FN_error = np.round(100 * FN_error, decimals=2)
    errors.append(error)
    FP_errors.append(FP_error)
    FN_errors.append(FN_error)
    n_barcodes.append(n_barcode)

sns.lineplot(n_std, errors, linewidth=5, label="Error")
sns.lineplot(n_std, FP_errors, linewidth=5, label="FP Error")
sns.lineplot(n_std, FN_errors, linewidth=5, label="FN Error")
plt.xticks(fontsize=20)
plt.yticks(
    fontsize=20,
)
plt.legend()
plt.show()

sns.lineplot(n_std, n_barcodes, linewidth=5)
plt.xticks(fontsize=20)
plt.yticks(
    fontsize=20,
)
plt.show()

#### Sources of error

There are around twice the number of false negatives (predicted to be a Dark GFP, but measured as bright) as there are false positives (predicted to be GFP, but measured as dark).

Some theories for these error classes:

False Positives:
    
    - Mutations in the promoter (should be constant within barcodes)
    
    - Strain variation (should be lower when averaging among strains)
    
    - Misread of barcodes
    
False Negatives:
    
    - Bleed from adjacent cells (should be corrected by averging among strains)
    
    - Multiple strains per trench (?)
    
    - Misread of barcodes

In [ ]:
plt.hist(
    called_df[called_df["Measured Dark GFP"] == True]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="grey",
    label="Measured Dark GFP",
)
plt.hist(
    called_df[called_df["Measured Dark GFP"] == False]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="green",
    label="Measured GFP",
)
plt.xlabel("90th Percentile Intensity", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
# plt.savefig("./2020-10-10_lDE11_figure_2.png",dpi=300,bbox_inches="tight")
plt.show()

plt.hist(
    called_df[called_df["Predicted Dark GFP"] == True]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="grey",
    alpha=0.7,
    label="Predicted Dark GFP",
)
plt.hist(
    called_df[called_df["Predicted Dark GFP"] == False]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="green",
    alpha=0.7,
    label="Predicted GFP",
)
plt.xlabel("90th Percentile Intensity", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
plt.savefig("./2020-11-07_lDE11_figure_1.png", dpi=300, bbox_inches="tight")
plt.show()

#### Averaging within barcodes

In [ ]:
barcode_median = called_df.groupby("Barcode").apply(
    lambda x: np.median(x["Measured GFP"])
)
barcode_median = pd.DataFrame(barcode_median, columns=["Measured GFP"]).reset_index()
barcode_median["Predicted Dark GFP"] = barcode_median.apply(
    lambda x: lDE11_lookup[x["Barcode"]], axis=1
)
barcode_median["Measured Dark GFP"] = barcode_median["Measured GFP"] < 2200

In [ ]:
barcode_median

In [ ]:
plt.hist(barcode_median["Measured GFP"], range=(0, 10000), bins=50)

In [ ]:
# plt.hist(barcode_median[barcode_median["Measured Dark GFP"]==True]["Measured GFP"],range=(0,10000),bins=50,color="grey",label="Measured Dark GFP")
# plt.hist(barcode_median[barcode_median["Measured Dark GFP"]==False]["Measured GFP"],range=(0,10000),bins=50,color="green",label="Measured GFP")
# plt.xlabel("90th Percentile Intensity",fontsize=20)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# plt.legend(fontsize=20)
# plt.savefig("./2020-10-10_lDE11_figure_2.png",dpi=300,bbox_inches="tight")
# # plt.show()

plt.hist(
    barcode_median[barcode_median["Predicted Dark GFP"] == True]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="grey",
    alpha=0.7,
    label="Predicted Dark GFP",
)
plt.hist(
    barcode_median[barcode_median["Predicted Dark GFP"] == False]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="green",
    alpha=0.7,
    label="Predicted GFP",
)
plt.xlabel("Median 90th Percentile Intensity", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
plt.savefig("./2020-10-10_lDE11_figure_4.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
TP = np.sum(
    (barcode_median["Predicted Dark GFP"] == False)
    & (barcode_median["Measured Dark GFP"] == False)
)
TN = np.sum(
    (barcode_median["Predicted Dark GFP"] == True)
    & (barcode_median["Measured Dark GFP"] == True)
)
FP = np.sum(
    (barcode_median["Predicted Dark GFP"] == False)
    & (barcode_median["Measured Dark GFP"] == True)
)
FN = np.sum(
    (barcode_median["Predicted Dark GFP"] == True)
    & (barcode_median["Measured Dark GFP"] == False)
)

In [ ]:
print(TP, TN)

In [ ]:
print(FP, FN)

In [ ]:
len(barcode_median)

In [ ]:
error = (FP + FN) / (TP + TN + FP + FN)
print("Error of median: " + str(error))
print("FP error: " + str(FP / (TP + TN + FP + FN)))
print("FN error: " + str(FN / (TP + TN + FP + FN)))

Error without averaging:

- Error: 0.11947107949273121
- FP error: 0.045159294772656976
- FN error: 0.07431178472007423

Error when taking median among strains:

- Error of median: 0.05637624359360868
- FP error: 0.017485679831172746
- FN error: 0.03889056376243594

Significantly cuts the false positive rate and false negative rates:
    
- Strain variation (should be lower when averaging among strains)
- Bleed from adjacent cells (should be corrected by averging among strains)

#### Strain frequency distribution

In [ ]:
num_barcodes = called_df.groupby("Reassigned Barcode").apply(lambda x: len(x))

In [ ]:
plt.hist(num_barcodes, range=(0, 20), bins=20)
plt.xlabel("# Appearances", fontsize=20)
plt.xticks(ticks=np.array(list(range(20))) + 0.5, labels=list(range(20)), fontsize=20)
plt.yticks(fontsize=20)
# plt.savefig("./2020-10-10_lDE11_figure_5.png",dpi=300,bbox_inches="tight")
plt.show()

In [ ]:
len(num_barcodes)

In [ ]:
np.mean(num_barcodes)

#### Hamming Distance Aware Barcodes

In [ ]:
lDE11_nanopore = pd.read_csv("./lDE11_final_df.tsv", delimiter="\t")
barcodes_list = lDE11_nanopore["barcode"].unique()
bit_arr = np.array([list(item) for item in barcodes_list]).astype(int)
both_on = bit_arr @ bit_arr.T
both_off = (-bit_arr + 1) @ (-bit_arr.T + 1)
ttl_match = both_on + both_off
np.fill_diagonal(ttl_match, 100)
closest_match = np.min(ttl_match, axis=0)
good_barcodes = barcodes_list[closest_match > 2]
lDE11_nanopore_good_barcodes = lDE11_nanopore[
    lDE11_nanopore["barcode"].isin(good_barcodes)
]

In [ ]:
lDE11_lookup = {}
for _, row in lDE11_nanopore_good_barcodes.iterrows():
    lDE11_lookup[barcode_to_FISH(row["barcode"])] = row["dark_gfp"]

In [ ]:
assign_strs = np.apply_along_axis(
    lambda x: "".join(x.astype(int).astype(str)), 0, assign_arr
)
barcode_df = pd.DataFrame(high_signal_barcodes_series, columns=["Barcode Signal"])
barcode_df["Barcode"] = assign_strs
barcode_df = barcode_df.reset_index(drop=False)
barcode_df["Bool Barcode"] = barcode_df.apply(
    lambda x: str_to_bool(x["Barcode"]), axis=1
)

In [ ]:
lDE11_lookup_bool = np.array(
    [str_to_bool(item) for item in list(lDE11_lookup.keys())]
).astype(int)
query_arr = np.array(barcode_df["Bool Barcode"].tolist()).astype(int)
both_on = query_arr @ lDE11_lookup_bool.T
both_off = (-query_arr + 1) @ (-lDE11_lookup_bool.T + 1)
ttl_match = both_on + both_off
best_match = np.max(ttl_match, axis=1)
arg_best_match = np.argmax(ttl_match, axis=1)
match_mask = best_match > 22
lDE11_lookup_strs = np.apply_along_axis(lambda x: bool_to_str(x), 1, lDE11_lookup_bool)
assigned_strs = lDE11_lookup_strs[arg_best_match]
barcode_df["Reassigned Barcode"] = assigned_strs
barcode_df["Reassigned Barcode"][~match_mask] = barcode_df["Barcode"]

In [ ]:
barcode_df["Predicted Dark GFP"] = barcode_df.apply(
    lambda x: lDE11_lookup[x["Reassigned Barcode"]]
    if x["Reassigned Barcode"] in lDE11_lookup.keys()
    else "Unknown",
    axis=1,
)

In [ ]:
ttl_true = np.sum([item == True for item in barcode_df["Predicted Dark GFP"].tolist()])
ttl_false = np.sum(
    [item == False for item in barcode_df["Predicted Dark GFP"].tolist()]
)
ttl_none = np.sum(
    [item == "Unknown" for item in barcode_df["Predicted Dark GFP"].tolist()]
)
ttl_called = ttl_true + ttl_false
ttl_trenches = len(
    kymograph_metadata.index.get_level_values("trenchid").unique().tolist()
)
ttl_trenches_with_cells = len(barcode_df)
percent_called = ttl_called / ttl_trenches
percent_called_w_cells = ttl_called / ttl_trenches_with_cells

In [ ]:
percent_called

In [ ]:
percent_called_w_cells

In [ ]:
headpath = "/home/de64/scratch/de64/2020-11-07_lDE11/2020-11-07_lDE11_GFP"
gfp_metadata = pd.read_parquet(headpath + "/percentiles")
gfp_groupby = gfp_metadata.groupby("trenchid")
median_GFP = gfp_groupby.apply(lambda x: np.median(x["FITC 90th Percentile"]))
gfp_metadata = gfp_metadata[gfp_metadata["timepoints"] == 18]
gfp_metadata = gfp_metadata.reset_index(drop=True)
gfp_metadata = gfp_metadata.set_index(["trenchid"], drop=True)
gfp_metadata["Median FITC 90th Percentile"] = median_GFP
gfp_metadata["Dark GFP"] = gfp_metadata["Median FITC 90th Percentile"] < 1700
gfp_signal = gfp_metadata["Median FITC 90th Percentile"]

In [ ]:
kymo_first_tpt = kymograph_metadata.loc[(slice(None), 0), :]
trenchid_map = get_trenchid_map(kymo_first_tpt, gfp_metadata)

In [ ]:
barcode_df["Measured Dark GFP"] = barcode_df.apply(
    lambda x: gfp_metadata.loc[trenchid_map[x["trenchid"]]]["Dark GFP"], axis=1
)
barcode_df["Measured GFP"] = barcode_df.apply(
    lambda x: gfp_metadata.loc[trenchid_map[x["trenchid"]]][
        "Median FITC 90th Percentile"
    ],
    axis=1,
)
called_df = barcode_df[barcode_df["Predicted Dark GFP"] != "Unknown"]

In [ ]:
ttl_correct = np.sum(called_df["Predicted Dark GFP"] == called_df["Measured Dark GFP"])
ttl_called = len(called_df)
print("Percent Correct:" + str(ttl_correct / ttl_called))

In [ ]:
TP = np.sum(
    (called_df["Predicted Dark GFP"] == False)
    & (called_df["Measured Dark GFP"] == False)
)
TN = np.sum(
    (called_df["Predicted Dark GFP"] == True) & (called_df["Measured Dark GFP"] == True)
)
FP = np.sum(
    (called_df["Predicted Dark GFP"] == False)
    & (called_df["Measured Dark GFP"] == True)
)
FN = np.sum(
    (called_df["Predicted Dark GFP"] == True)
    & (called_df["Measured Dark GFP"] == False)
)

In [ ]:
error = (FP + FN) / (TP + TN + FP + FN)
print("Error: " + str(error))
print("FP error: " + str(FP / (TP + TN + FP + FN)))
print("FN error: " + str(FN / (TP + TN + FP + FN)))

In [ ]:
called_df.groupby("Reassigned Barcode").apply(lambda x: len(x))

In [ ]:
plt.hist(
    called_df[called_df["Measured Dark GFP"] == True]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="grey",
    label="Measured Dark GFP",
)
plt.hist(
    called_df[called_df["Measured Dark GFP"] == False]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="green",
    label="Measured GFP",
)
plt.xlabel("90th Percentile Intensity", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
# plt.savefig("./2020-10-10_lDE11_figure_2.png",dpi=300,bbox_inches="tight")
plt.show()

plt.hist(
    called_df[called_df["Predicted Dark GFP"] == True]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="grey",
    alpha=0.7,
    label="Predicted Dark GFP",
)
plt.hist(
    called_df[called_df["Predicted Dark GFP"] == False]["Measured GFP"],
    range=(0, 10000),
    bins=50,
    color="green",
    alpha=0.7,
    label="Predicted GFP",
)
plt.xlabel("90th Percentile Intensity", fontsize=20)
plt.ylabel("Lineages", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
plt.savefig("./2020-11-07_lDE11_figure_1.png", dpi=300, bbox_inches="tight")

plt.show()

### Figures for retreat

In [ ]:
import h5py
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
headpath = "/home/de64/scratch/de64/2020-11-07_lDE11/2020-11-07_lDE11"

In [ ]:
meta = tr.pandas_hdf5_handler(headpath + "/metadata.hdf5")
global_df = meta.read_df("global", read_metadata=True)
channels = global_df.metadata["channels"]
selected_df = global_df.loc[20]

In [ ]:
def get_selected_imgs(selected_df, channels, headpath):
    img_dict = {channel: [] for channel in channels}
    for row in selected_df.iterrows():
        file_idx = int(row[1]["File Index"])
        img_idx = int(row[1]["Image Index"])
        with h5py.File(
            headpath + "/hdf5/hdf5_" + str(file_idx) + ".hdf5", "r"
        ) as infile:
            for channel in channels:
                img_dict[channel].append(infile[channel][img_idx])
    img_dict = {key: np.array(val) for key, val in img_dict.items()}
    return img_dict

In [ ]:
img_dict = get_selected_imgs(selected_df, channels, headpath)

In [ ]:
test = img_dict["Cy5"]

In [ ]:
channels

In [ ]:
grid_channels = ["RFP", "Cy5", "Cy7"]
# grid_colors = ["Reds","Purples","Greys_r"]
num_cycles = 8

vrange_list = {
    key: (np.percentile(val.flatten(), 0), np.percentile(val.flatten(), 99.99))
    for key, val in img_dict.items()
}


fig, axes = plt.subplots(len(grid_channels), num_cycles)
for i in range(num_cycles):
    for j, channel in enumerate(grid_channels):
        axes[j, i].imshow(
            img_dict[channel][i][275:375, 1010:1110],
            "Greys_r",
            vmin=vrange_list[channel][0],
            vmax=vrange_list[channel][1],
        )
        axes[j, i].axis("off")
fig.set_size_inches(20, 7.5)
plt.tight_layout()
plt.show()

In [ ]:
import tifffile

In [ ]:
selected_channels = ["RFP", "Cy5", "Cy7"]
with tifffile.TiffWriter("./lDE11_stack.tiff", "w") as stack:
    for channel in selected_channels:
        for i in range(img_dict[channel].shape[0]):
            vmin = vrange_list[channel][0]
            vmax = vrange_list[channel][1]
            val = (img_dict[channel] - vmin) / (vmax - vmin)
            arr = val[i, 275:425]
            stack.save(arr, contiguous=True)

In [ ]:
headpath = "/home/de64/scratch/de64/2020-11-07_lDE11/2020-11-07_lDE11_GFP"

In [ ]:
meta = tr.pandas_hdf5_handler(headpath + "/metadata.hdf5")
global_df = meta.read_df("global", read_metadata=True)
channels = global_df.metadata["channels"]
selected_df = global_df.loc[20]

In [ ]:
img_dict = get_selected_imgs(selected_df, channels, headpath)

In [ ]:
img_dict = get_selected_imgs(selected_df, ["mCherry"], headpath)

In [ ]:
img_dict["FITC"].shape

In [ ]:
with tifffile.TiffWriter("./lDE11_mCherry.tiff", "w") as outfile:
    outfile.save(img_dict["mCherry"][:, 275:425], contiguous=True)

In [ ]:
headpath = "/home/de64/scratch/de64/2020-11-07_lDE11/2020-11-07_lDE11"

In [ ]:
kymograph_metadata = pd.read_parquet(headpath + "/kymograph/metadata")
gfp_kymograph_metadata = pd.read_parquet(
    "/home/de64/scratch/de64/2020-11-07_lDE11/2020-11-07_lDE11_GFP/kymograph/metadata"
)
kymograph_metadata = kymograph_metadata.set_index(["trenchid", "timepoints"])
gfp_kymograph_metadata = gfp_kymograph_metadata.set_index(["trenchid"])

In [ ]:
kymo_first_tpt = kymograph_metadata.loc[(slice(None), 0), :]
trenchid_map = get_trenchid_map(kymo_first_tpt, gfp_kymograph_metadata)
inv_trenchid_map = {val: key for key, val in trenchid_map.items()}

In [ ]:
inv_trenchid_map

In [ ]:
selected_df = kymograph_metadata.loc[100][0:1]

In [ ]:
def get_selected_kymo(selected_df, channels, headpath):
    img_dict = {channel: [] for channel in channels}

    for row in selected_df.iterrows():
        file_idx = int(row[1]["File Index"])
        trench_idx = int(row[1]["File Trench Index"])
        with h5py.File(
            headpath + "/kymograph/kymograph_" + str(file_idx) + ".hdf5", "r"
        ) as infile:
            for channel in channels:
                img_dict[channel].append(infile[channel][trench_idx])
    img_dict = {key: np.array(val) for key, val in img_dict.items()}
    return img_dict


def plot_barcode(img_dict):
    vrange_list = {
        key: (np.percentile(val.flatten(), 10), np.percentile(val.flatten(), 99))
        for key, val in img_dict.items()
    }
    rescaled_arrs = {
        key: (
            (img_dict[key][0] - vrange_list[key][0])
            / (vrange_list[key][1] - vrange_list[key][0])
        )
        for key, val in img_dict.items()
    }

    barcode_kymo = np.concatenate(
        [rescaled_arrs["RFP"], rescaled_arrs["Cy5"], rescaled_arrs["Cy7"]], axis=0
    )
    ky_handle = tr.kymo_handle()
    ky_handle.import_wrap(barcode_kymo)
    plt.imshow(ky_handle.return_unwrap(), "Greys_r")

In [ ]:
selected_df = kymograph_metadata.loc[5200:5201][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
selected_df = kymograph_metadata.loc[12000:12001][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
selected_df = kymograph_metadata.loc[13300:13301][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
selected_df = kymograph_metadata.loc[13500:13501][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
selected_df = kymograph_metadata.loc[13810:13811][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
selected_df = kymograph_metadata.loc[4578:4579][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
selected_df = kymograph_metadata.loc[4578:4579][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
mapped_gfp = gfp_kymograph_metadata.loc[
    inv_trenchid_map[selected_df.index.get_level_values("trenchid")[0]]
]
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2020-10-10_lDE11_spectra_fixed/2020-10-10_lDE11_spectra_fixed_GFP/kymograph/kymograph_"
    + str(mapped_gfp["File Index"])
    + ".hdf5",
    "r",
) as infile:
    data = infile["FITC"][mapped_gfp["File Trench Index"], 0]
plt.imshow(data, "Greys_r", vmin=0, vmax=5000)

In [ ]:
selected_df = kymograph_metadata.loc[1561:1562][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
mapped_gfp = gfp_kymograph_metadata.loc[
    inv_trenchid_map[selected_df.index.get_level_values("trenchid")[0]]
]
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2020-10-10_lDE11_spectra_fixed/2020-10-10_lDE11_spectra_fixed_GFP/kymograph/kymograph_"
    + str(mapped_gfp["File Index"])
    + ".hdf5",
    "r",
) as infile:
    data = infile["FITC"][mapped_gfp["File Trench Index"], 0]
plt.imshow(data, "Greys_r", vmin=0, vmax=5000)

In [ ]:
selected_df = kymograph_metadata.loc[1551:1552][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
mapped_gfp = gfp_kymograph_metadata.loc[
    inv_trenchid_map[selected_df.index.get_level_values("trenchid")[0]]
]
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2020-10-10_lDE11_spectra_fixed/2020-10-10_lDE11_spectra_fixed_GFP/kymograph/kymograph_"
    + str(mapped_gfp["File Index"])
    + ".hdf5",
    "r",
) as infile:
    data = infile["FITC"][mapped_gfp["File Trench Index"], 0]
plt.imshow(data, "Greys_r", vmin=0, vmax=5000)

In [ ]:
selected_df = kymograph_metadata.loc[1845:1846][0:1]
img_dict = get_selected_kymo(selected_df, ["RFP", "Cy5", "Cy7"], headpath)
plot_barcode(img_dict)

In [ ]:
mapped_gfp = gfp_kymograph_metadata.loc[
    inv_trenchid_map[selected_df.index.get_level_values("trenchid")[0]]
]
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2020-10-10_lDE11_spectra_fixed/2020-10-10_lDE11_spectra_fixed_GFP/kymograph/kymograph_"
    + str(mapped_gfp["File Index"])
    + ".hdf5",
    "r",
) as infile:
    data = infile["FITC"][mapped_gfp["File Trench Index"], 0]
plt.imshow(data, "Greys_r", vmin=0, vmax=5000)

In [ ]:
selected_df

In [ ]:
grid_channels = ["RFP", "Cy5", "Cy7"]
# grid_colors = ["Reds","Purples","Greys_r"]
num_cycles = 8

vrange_list = {
    key: (np.percentile(val.flatten(), 0), np.percentile(val.flatten(), 99.99))
    for key, val in img_dict.items()
}

fig, axes = plt.subplots(len(grid_channels), num_cycles)
for i in range(num_cycles):
    for j, channel in enumerate(grid_channels):
        axes[j, i].imshow(
            img_dict[channel][i][225:325, 1010:1110],
            "Greys_r",
            vmin=vrange_list[channel][0],
            vmax=vrange_list[channel][1],
        )
        axes[j, i].axis("off")
fig.set_size_inches(20, 7.5)
plt.tight_layout()
plt.show()